<a href="https://colab.research.google.com/github/Rex-Chan99/Bitcoin_prediction/blob/main/Master_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Packages

In [213]:
!pip install pandas_ta
!pip install numpy==1.23.5
!pip install pandas
!pip install seaborn
!pip install xgboost

In [214]:
# Importing libraries for data manipulation
import numpy as np
import pandas as pd

# Importing libraries for data visualization
import seaborn as sns
import matplotlib.pyplot as plt

# Importing libraries for scaling the data
from sklearn.preprocessing import MinMaxScaler

# Importing libraries for building linear regression model
import statsmodels.api as sm
import statsmodels.formula.api as smf

from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.outliers_influence import summary_table, OLSInfluence
from statsmodels.regression.linear_model import OLSResults
from statsmodels.stats.stattools import durbin_watson as dwtest
from statsmodels.sandbox.stats.runs import runstest_1samp
from statsmodels.stats.diagnostic import het_white

from statsmodels.compat import lzip

import statsmodels.stats.api as sms

from scipy import stats
from scipy.stats import zscore

from sklearn import linear_model
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, RidgeCV, LassoCV, ElasticNetCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV, RepeatedKFold, cross_val_score

import pylab as py

# To help with model building
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import (
    BaggingRegressor,
    RandomForestRegressor,
    GradientBoostingRegressor,
    AdaBoostRegressor,
    StackingRegressor,
)
from xgboost import XGBRegressor

# To tune model, get different metric scores, and split data
from sklearn import metrics
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score

# To be used for data scaling and one hot encoding
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder

# To impute missing values
from sklearn.impute import SimpleImputer

# To do hyperparameter tuning
from sklearn.model_selection import RandomizedSearchCV

# To be used for creating pipelines and personalizing them
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# To ignore warnings
import warnings
warnings.filterwarnings("ignore")

# To suppress scientific notations
pd.set_option("display.float_format", lambda x: "%.4f" % x)

# To define maximum number of columns to be displayed in a dataframe
pd.set_option("display.max_columns", None)


#Load dataset

In [215]:
!git clone https://github.com/ArthurChan-1111/Bitcoin_prediction.git

%cd Bitcoin_prediction

%pip install pandas_ta

import pandas as pd
import pandas_ta as ta
import csv
import numpy as np

btc_data = pd.read_csv('Bitcoin Historical Data.csv', on_bad_lines='skip', lineterminator='\n')
btc_data.tail(5)

Cloning into 'Bitcoin_prediction'...
remote: Enumerating objects: 341, done.
remote: Counting objects: 100% (101/101), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 341 (delta 64), reused 25 (delta 13), pack-reused 240 (from 2)
Receiving objects: 100% (341/341), 38.43 MiB | 20.56 MiB/s, done.
Resolving deltas: 100% (170/170), done.
/content/Bitcoin_prediction/Bitcoin_prediction/Bitcoin_prediction/Bitcoin_prediction


,Date,Price,Open,High,Low,Vol. ('000),Change %\r
4107,1/5/2014,1014.7000,924.7000,1029.9000,911.4000,21.37,9.74%\r
4108,1/4/2014,924.7000,884.3000,932.2000,848.3000,14.24,4.57%\r
4109,1/3/2014,884.3000,856.9000,888.2000,839.4000,9.71,3.19%\r
4110,1/2/2014,856.9000,815.9000,886.2000,810.5000,12.81,5.02%\r
4111,1/1/2014,815.9000,805.9000,829.9000,771.0000,10.76,1.24%


In [216]:
# Convert the dictionary to a DataFrame
btc_data = pd.DataFrame(btc_data)

# Parse the "Date" column into datetime format
btc_data["Date"] = pd.to_datetime(btc_data["Date"], format="%m/%d/%Y")

# Sort the data by date in ascending order
btc_data.sort_values(by="Date", ascending=True, inplace=True)


In [217]:
gold_price_data = pd.read_csv('gold_price_data.csv', on_bad_lines='skip', lineterminator='\n', sep=';')
gold_price_data.tail(5)

,Date,Gold_Price,Gold_Volume
4047,2025-01-30,2794.0600,83835.0000
4048,2025-01-31,2799.2300,91164.0000
4049,2025-02-01,2799.2300,91164.0000
4050,2025-02-02,2799.2300,91164.0000
4051,2025-02-03,2794.4100,1631.0000


In [218]:
# rename the column of Gold_Volume\r to Gold_Volume
gold_price_data.rename(columns={'Gold_Volume\r': 'Gold_Volume'}, inplace=True)

In [219]:
risk_free_data = pd.read_csv('risk_free_data.csv', on_bad_lines='skip', lineterminator='\n', sep=',')
risk_free_data.tail(5)

,Date,US_10Y,Yield_Spread
4126,2025-04-19,4.3400,0.5300
4127,2025-04-20,4.3400,0.5300
4128,2025-04-21,4.4200,0.6700
4129,2025-04-22,4.4100,0.6500
4130,2025-04-23,4.4000,0.5900


In [220]:
# rename the Yield_Spread\r to Yield_Spread
risk_free_data.rename(columns={'Yield_Spread\r': 'Yield_Spread'}, inplace=True)

In [221]:
spy_data = pd.read_csv('spy_data.csv', on_bad_lines='skip', lineterminator='\n', sep=',')
spy_data.tail(5)

,Date,SPY_Price,SPY_Volume
4114,2025-04-07,504.3800,256611400.0000
4115,2025-04-08,496.4800,165816600.0000
4116,2025-04-09,548.6200,241867300.0000
4117,2025-04-10,524.5800,162331200.0000
4118,2025-04-11,533.9400,97741700.0000


In [222]:
#Change the 'Date' to datetime format
gold_price_data['Date'] = pd.to_datetime(gold_price_data['Date'])
risk_free_data['Date'] = pd.to_datetime(risk_free_data['Date'])
spy_data['Date'] = pd.to_datetime(spy_data['Date'])

#Data Preprocessing

In [223]:
#Function to clean and convert volume data
def clean_volume(volume):
    if isinstance(volume, str):  # Check if the value is a string
        volume = volume.replace(',', '')  # Remove commas
        if 'B' in volume:  # If the value contains 'B' (billions)
            return float(volume.replace('B', '')) * 1_000_000_000
        elif 'M' in volume:  # If the value contains 'M' (millions)
            return float(volume.replace('M', '')) * 1_000_000
        elif 'K' in volume:  # If the value contains 'K' (thousands)
            return float(volume.replace('K', '')) * 1_000
        else:  # If no suffix is present, convert to float directly
            return float(volume)
    return np.nan  # Handle unexpected cases

# Apply the cleaning function to the volume column
btc_data["Volume"] = btc_data["Vol. ('000)"].apply(clean_volume)
btc_data = btc_data.drop("Vol. ('000)", axis=1)

# 4. Remove '%' from "Change %" and convert to numeric
btc_data.rename(columns={"Change %\r": "Change"}, inplace=True)
btc_data["Change"] = btc_data["Change"].str.replace("%", "").str.strip().astype(float)

In [224]:
# Define the missing data as a dictionary
missing_data = {
    'Date': '01/12/2025',
    'Price': 94541.8,
    'Open': 94600.0,
    'High': 95384.3,
    'Low': 93711.2,
    'Volume': 17.60,  # Assuming 'Vol. (\'000)' is in thousands
    'Change': -0.07  # Assuming 'Change %' is already converted to a float
}

# Convert the dictionary to a DataFrame
missing_data_df = pd.DataFrame([missing_data])

# Ensure the 'Date' column is in datetime format
missing_data_df['Date'] = pd.to_datetime(missing_data_df['Date'])

# Append the missing data to btc_data
btc_data = pd.concat([btc_data, missing_data_df], ignore_index=True)

# Filter the data for the date range 01/10/2025 to 01/20/2025
filtered_data = btc_data[(btc_data['Date'] >= '2025-01-10') & (btc_data['Date'] <= '2025-01-20')]

# Display the filtered data
print(filtered_data)

           Date       Price        Open        High         Low  Change  \
4027 2025-01-10  94724.1000  92540.5000  95760.6000  92310.6000  2.3600   
4028 2025-01-11  94612.0000  94724.0000  94765.5000  93847.8000 -0.1200   
4029 2025-01-13  94534.3000  94540.5000  95906.9000  89664.8000 -0.0800   
4030 2025-01-14  96542.1000  94536.7000  97361.2000  94359.5000  2.1200   
4031 2025-01-15 100499.8000  96547.4000 100671.0000  96501.9000  4.1000   
4032 2025-01-16 100004.5000 100499.9000 100812.4000  97400.2000 -0.4900   
4033 2025-01-17 104084.9000  99991.9000 105843.8000  99956.2000  4.0800   
4034 2025-01-18 104536.9000 104084.7000 104953.1000 102297.0000  0.4300   
4035 2025-01-19 101319.5000 104536.8000 106338.2000  99742.7000 -3.0800   
4036 2025-01-20 102197.7000 101313.3000 109228.6000  99676.0000  0.8700   
4112 2025-01-12  94541.8000  94600.0000  95384.3000  93711.2000 -0.0700   

       Volume  
4027 125.4200  
4028  72.6600  
4029 115.2000  
4030  87.7900  
4031  95.3200  
403

In [225]:
# Day of the week (0=Monday, 6=Sunday)
btc_data["Day_of_Week"] = btc_data["Date"].dt.dayofweek

# Week of the year
btc_data["Week_of_Year"] = btc_data["Date"].dt.isocalendar().week

# Month of the year
btc_data["Month"] = btc_data["Date"].dt.month

# Quarter of the year
btc_data["Quarter"] = btc_data["Date"].dt.quarter

# Year
btc_data["Year"] = btc_data["Date"].dt.year

## Signal Detection

In [226]:
# Calculate RSI, EMA, SMA, and MACD
btc_data["RSI_6"] = ta.rsi(btc_data["Price"], length=6)  # Relative Strength Index
btc_data["RSI_12"] = ta.rsi(btc_data["Price"], length=12)  # Relative Strength Index
btc_data["EMA_14"] = ta.ema(btc_data["Price"], length=14)  # Exponential Moving Average
btc_data["SMA_14"] = ta.sma(btc_data["Price"], length=14)  # Simple Moving Average

# Add On-Balance Volume (OBV)
btc_data["OBV"] = ta.obv(btc_data["Price"], btc_data["Volume"])

# Calculate MACD --------------------------------------------------------------------------------------
macd = ta.macd(btc_data["Price"], fast=12, slow=26, signal=9)
btc_data["MACD"] = macd["MACD_12_26_9"]
btc_data["MACD_Signal"] = macd["MACDs_12_26_9"]

# Create MACD_buy: 1 if MACD crosses above MACD_Signal, otherwise 0
btc_data["MACD_buy"] = 0  # Default to 0 (no buy signal)
btc_data.loc[
    (btc_data["MACD"] > btc_data["MACD_Signal"]) &  # MACD is above Signal
    (btc_data["MACD"].shift(1) <= btc_data["MACD_Signal"].shift(1)),  # Previous MACD was below or equal to Signal
    "MACD_buy"] = 1  # Buy signal

# Create MACD_sell: 1 if MACD crosses below MACD_Signal, otherwise 0
btc_data["MACD_sell"] = 0  # Default to 0 (no sell signal)
btc_data.loc[
    (btc_data["MACD"] <= btc_data["MACD_Signal"]) &  # MACD is below or equal to Signal
    (btc_data["MACD"].shift(1) > btc_data["MACD_Signal"].shift(1)),  # Previous MACD was above Signal
    "MACD_sell"] = 1  # Sell signal

# New Variable: MACD_above_signal (1 if MACD > MACD_Signal, otherwise 0)
btc_data["MACD_above_signal"] = (btc_data["MACD"] > btc_data["MACD_Signal"]).astype(int)

# New Variable: MACD_below_signal (1 if MACD <= MACD_Signal, otherwise 0)
btc_data["MACD_below_signal"] = (btc_data["MACD"] <= btc_data["MACD_Signal"]).astype(int)

# Create MACD_cum_buy: Accumulate consecutive days where MACD_above_signal is 1
btc_data["MACD_cum_buy"] = (
    btc_data["MACD_above_signal"]
    .groupby((btc_data["MACD_above_signal"] == 0).cumsum())  # Group by resets when MACD_above_signal is 0
    .cumcount()  # Count consecutive days starting from 1
)
btc_data.loc[btc_data["MACD_above_signal"] == 0, "MACD_cum_buy"] = 0  # Reset to 0 where the condition is not met

# Create MACD_cum_sell: Accumulate consecutive days where MACD_below_signal is 1
btc_data["MACD_cum_sell"] = (
    btc_data["MACD_below_signal"]
    .groupby((btc_data["MACD_below_signal"] == 0).cumsum())  # Group by resets when MACD_below_signal is 0
    .cumcount()  # Count consecutive days starting from 1
)
btc_data.loc[btc_data["MACD_below_signal"] == 0, "MACD_cum_sell"] = 0  # Reset to 0


# Calculate Bollinger Bands --------------------------------------------------------------------------------------
bb = ta.bbands(btc_data["Price"], length=20, std=2)  # 20-period BB with 2 standard deviations
btc_data["BB_upper"] = bb.get("BBU_20_2.0")  # Upper Bollinger Band
btc_data["BB_lower"] = bb.get("BBL_20_2.0")  # Lower Bollinger Band

# Create BB_Buy: 1 if Price crosses below the Lower Band, otherwise 0
btc_data["BB_Buy"] = 0  # Default to 0 (no buy signal)
btc_data.loc[
    (btc_data["Price"] < btc_data["BB_lower"]) &  # Price is below the Lower Band
    (btc_data["Price"].shift(1) >= btc_data["BB_lower"].shift(1)),  # Previous Price was above or equal to Lower Band
    "BB_Buy"] = 1  # Buy signal

# Create BB_Sell: 1 if Price crosses above the Upper Band, otherwise 0
btc_data["BB_Sell"] = 0  # Default to 0 (no sell signal)
btc_data.loc[
    (btc_data["Price"] > btc_data["BB_upper"]) &  # Price is above the Upper Band
    (btc_data["Price"].shift(1) <= btc_data["BB_upper"].shift(1)),  # Previous Price was below or equal to Upper Band
    "BB_Sell"] = 1  # Sell signal

# New Variable: Price_above_lower_band (1 if Price > BB_lower, otherwise 0)
btc_data["Price_below_BB_lower"] = (btc_data["Price"] < btc_data["BB_lower"]).astype(int)

# New Variable: Price_below_upper_band (1 if Price < BB_upper, otherwise 0)
btc_data["Price_above_BB_upper"] = (btc_data["Price"] > btc_data["BB_upper"]).astype(int)

# Create BB_cum_Buy: Accumulate consecutive days where Price_below_BB_lower is 1
btc_data["BB_cum_Buy"] = (
    btc_data["Price_below_BB_lower"]
    .groupby((btc_data["Price_below_BB_lower"] == 0).cumsum())  # Group by resets when Price_below_BB_lower is 0
    .cumcount()  # Count consecutive days starting from 1
)
btc_data.loc[btc_data["Price_below_BB_lower"] == 0, "BB_cum_Buy"] = 0  # Reset to 0 where the condition is not met

# Create BB_cum_Sell: Accumulate consecutive days where Price_above_BB_upper is 1
btc_data["BB_cum_Sell"] = (
    btc_data["Price_above_BB_upper"]
    .groupby((btc_data["Price_above_BB_upper"] == 0).cumsum())  # Group by resets when Price_above_BB_upper is 0
    .cumcount()  # Count consecutive days starting from 1
)
btc_data.loc[btc_data["Price_above_BB_upper"] == 0, "BB_cum_Sell"] = 0  # Reset to 0 where the condition is not met

# Calculate ATR --------------------------------------------------------------------------------------
btc_data["ATR"] = ta.atr(btc_data["High"], btc_data["Low"], btc_data["Price"], length=14)

# Set Stop-Loss Levels (Example with Long Trade)
atr_multiplier = 2
btc_data["Stop_Loss_Long"] = btc_data["Price"] - (btc_data["ATR"] * atr_multiplier) #Stop-Loss for Buy (Long) Trade
btc_data["Stop_Loss_Short"] = btc_data["Price"] + (btc_data["ATR"] * atr_multiplier) #Stop-Loss for Sell (Short) Trade

# Calculate VWAP --------------------------------------------------------------------------------------
btc_data.set_index("Date", inplace=True) # Set the "Date" column as the index
btc_data["VWAP"] = ta.vwap(btc_data["High"], btc_data["Low"], btc_data["Price"], btc_data["Volume"])

# Create VWAP_Buy: 1 if Price crosses above VWAP, otherwise 0
btc_data["VWAP_Buy"] = 0  # Default to 0 (no buy signal)
btc_data.loc[
    (btc_data["Price"] > btc_data["VWAP"]) &  # Price is above VWAP
    (btc_data["Price"].shift(1) <= btc_data["VWAP"].shift(1)),  # Previous Price was below or equal to VWAP
    "VWAP_Buy"] = 1  # Buy signal

# Create VWAP_Sell: 1 if Price crosses below VWAP, otherwise 0
btc_data["VWAP_Sell"] = 0  # Default to 0 (no sell signal)
btc_data.loc[
    (btc_data["Price"] < btc_data["VWAP"]) &  # Price is below VWAP
    (btc_data["Price"].shift(1) >= btc_data["VWAP"].shift(1)),  # Previous Price was above or equal to VWAP
    "VWAP_Sell"] = 1  # Sell signal

## Fear and greed index
#fear_and_greed_index = pd.read_csv('fear_and_greed_index.csv')
## rename the date column in fear_and_greed_index to Date
#fear_and_greed_index.rename(columns={"date": "Date", "value": "Fear_and_Greed_Index"}, inplace=True)
## convert the date column to datetime
#fear_and_greed_index["Date"] = pd.to_datetime(fear_and_greed_index["Date"], format="%Y-%m-%d")
## dropping the timestamp, value_classification, time_until_update columns
#fear_and_greed_index.drop(columns=["timestamp", "value_classification", "time_until_update"], inplace=True)
#fear_and_greed_index.head()

## Join the two dataframes on the "Date" column, dropping
#btc_data = btc_data.merge(fear_and_greed_index, on="Date", how="left")
#btc_data.head(20)

# Support and Resistance Levels
btc_data["Support"] = btc_data["Low"].rolling(window=20).min()  # Lowest low in the past 20 days
btc_data["Resistance"] = btc_data["High"].rolling(window=20).max()  # Highest high in the past 20 days

## Add two columns for extreme fear and extreme greed
#btc_data["Extreme_Fear"] = np.where(btc_data["Fear_and_Greed_Index"] < 0.2, 1, 0)
#btc_data["Extreme_Greed"] = np.where(btc_data["Fear_and_Greed_Index"] > 0.8, 1, 0)
#btc_data.describe().T

# Use 70 in RSI to classify overbought and 30 to classify oversold
btc_data["RSI_12_Oversold"] = np.where(btc_data["RSI_12"] < 30, 1, 0)  # Create RSI_12_Oversold column
btc_data.loc[btc_data["RSI_12_Oversold"] == 0, "Consecutive_RSI_12_Oversold"] = 0  # Reset to 0 where the condition is not met
btc_data["RSI_12_Overbought"] = np.where(btc_data["RSI_12"] > 70, 1, 0)

# Adding RSI_6 greater than RSI_12 to show bullish divergence and conversely
btc_data["RSI_Divergence"] = np.where(btc_data["RSI_6"] > btc_data["RSI_12"], 1, 0)
btc_data.describe().T


## Count consecutive appearances of Extreme_Fear
#btc_data["Consecutive_Extreme_Fear"] = (
#    btc_data["Extreme_Fear"]
#    .groupby((btc_data["Extreme_Fear"] == 0).cumsum())  # Group by resets when Extreme_Fear is 0
#    .cumcount()  # Count consecutive days starting from 1
#btc_data.loc[btc_data["Extreme_Fear"] == 0, "Consecutive_Extreme_Fear"] = 0  # Reset to 0 where the condition is not met

## Count consecutive appearances of Extreme_Greed
#btc_data["Consecutive_Extreme_Greed"] = (
#    btc_data["Extreme_Greed"]
#    .groupby((btc_data["Extreme_Greed"] == 0).cumsum())  # Group by resets when Extreme_Greed is 0
#    .cumcount()  # Count consecutive days starting from 1
#)
#btc_data.loc[btc_data["Extreme_Greed"] == 0, "Consecutive_Extreme_Greed"] = 0  # Reset to 0 where the condition is not met

# Count consecutive appearances of RSI_12_Overbought
btc_data["Consecutive_RSI_12_Overbought"] = (
    btc_data["RSI_12_Overbought"]
    .groupby((btc_data["RSI_12_Overbought"] == 0).cumsum())  # Group by resets when RSI_12_Overbought is 0
    .cumcount()  # Count consecutive days starting from 1
)
btc_data.loc[btc_data["RSI_12_Overbought"] == 0, "Consecutive_RSI_12_Overbought"] = 0  # Reset to 0 where the condition is not met

# Count consecutive appearances of RSI_12_Oversold
btc_data["Consecutive_RSI_12_Oversold"] = (
    btc_data["RSI_12_Oversold"]
    .groupby((btc_data["RSI_12_Oversold"] == 0).cumsum())  # Group by resets when RSI_12_Oversold is 0
    .cumcount()  # Count consecutive days starting from 1
)

In [227]:
#Add continous variables

#ATR ratio
btc_data['ATR_Ratio'] = (btc_data['ATR'] / btc_data['Price']) * 100

# Calculate 50-day and 200-day moving averages
btc_data['MA50'] = btc_data['Price'].rolling(window=50).mean()
btc_data['MA200'] = btc_data['Price'].rolling(window=200).mean()

# Calculate Price relative to 50-day MA (%)
btc_data['Price_vs_MA50'] = ((btc_data['Price'] - btc_data['MA50']) / btc_data['MA50']) * 100

# Calculate Price relative to 200-day MA (%)
btc_data['Price_vs_MA200'] = ((btc_data['Price'] - btc_data['MA200']) / btc_data['MA200']) * 100

# Calculate MA Distance Ratio (Ratio of MA50 to MA200 distance)
btc_data['MA_Distance_Ratio'] = ((btc_data['MA50'] - btc_data['MA200']) / btc_data['Price']) * 100

In [228]:
from scipy.stats import linregress

# Ensure btc_data is sorted by Date
btc_data = btc_data.sort_values('Date')

# 1. Volume_Ratio: Current volume / 20-day avg volume
btc_data['Avg_Volume_20'] = btc_data['Volume'].rolling(window=20).mean()
btc_data['Volume_Ratio'] = btc_data['Volume'] / btc_data['Avg_Volume_20']

# 2. OBV_Slope: 5-day slope of On-Balance Volume (OBV)
# Calculate OBV
btc_data['OBV'] = (np.sign(btc_data['Price'].diff()) * btc_data['Volume']).fillna(0).cumsum()

# Calculate 5-day slope of OBV
def calculate_slope(series):
    x = np.arange(len(series))
    slope, _, _, _, _ = linregress(x, series)
    return slope

btc_data['OBV_Slope'] = btc_data['OBV'].rolling(window=5).apply(calculate_slope, raw=False)

In [229]:
# Calculate the percentage price change over the next 7-day and 2-day (Target Variable for Regression)
btc_data["7D_Pct_Change"] = ((btc_data["Price"].shift(-7) - btc_data["Price"]) / btc_data["Price"]) * 100
btc_data["2D_Pct_Change"] = ((btc_data["Price"].shift(-2) - btc_data["Price"]) / btc_data["Price"]) * 100
btc_data["14D_Pct_Change"] = ((btc_data["Price"].shift(-14) - btc_data["Price"]) / btc_data["Price"]) * 100
btc_data["30D_Pct_Change"] = ((btc_data["Price"].shift(-30) - btc_data["Price"]) / btc_data["Price"]) * 100

# Create binary variables for 7-day and 2-day percentage changes
btc_data["7D_Positive"] = (btc_data["7D_Pct_Change"] > 0).astype(int)  # 1 if positive, 0 if negative
btc_data["2D_Positive"] = (btc_data["2D_Pct_Change"] > 0).astype(int)  # 1 if positive, 0 if negative


## Pattern Detection

In [230]:
# Asending Triangle
# Sort data by Date
btc_data = btc_data.sort_values('Date')

# Calculate exponential moving average for Resistance_Level
btc_data['Resistance_Level'] = btc_data['High'].ewm(span=7, adjust=False).mean()

# Identify rising trendline (higher lows)
btc_data['Low_Shifted'] = btc_data['Low'].shift(1)
btc_data['Higher_Low'] = (btc_data['Low'] > btc_data['Low_Shifted']).astype(int)

# Detect breakout using rolling mean of Volume
btc_data['Breakout'] = btc_data['Price'] > btc_data['Resistance_Level']

# Add ascending triangle detection signal
btc_data['Ascending_Triangle_Breakout'] = btc_data['Breakout']

# Ensure Breakout is binary (1 or 0)
btc_data['Breakout'] = btc_data['Breakout'].astype(int)

# Ensure Ascending_Triangle_Breakout is binary (1 or 0)
btc_data['Ascending_Triangle_Breakout'] = btc_data['Ascending_Triangle_Breakout'].astype(int)

# Calculate the rolling mean of Volume over 7 days
btc_data['Avg_Volume'] = btc_data['Volume'].rolling(window=7).mean()

# Detect breakout with volume confirmation
btc_data['Breakout_With_Volume'] = ((btc_data['High'] > btc_data['Resistance_Level']) &
                                    (btc_data['Volume'] > btc_data['Avg_Volume'])).astype(int)

# Add ascending triangle detection signal with volume
btc_data['Ascending_Triangle_Breakout_With_Volume'] = btc_data['Breakout_With_Volume']

In [231]:
# Count the number of breakouts
breakout_count = btc_data['Ascending_Triangle_Breakout_With_Volume'].sum()
print(f"Number of breakouts: {breakout_count}")

Number of breakouts: 1069


In [232]:
btc_data['Triangle_Completion'] = btc_data['Higher_Low'].rolling(window=7).mean()  # Simulate completion as a rolling mean of Higher_Low
btc_data['Triangle_Height'] = ((btc_data['High'] - btc_data['Low']) / btc_data['Price']).rolling(window=7).mean()  # Height as % of price
btc_data['Triangle_Duration'] = btc_data['Higher_Low'].rolling(window=7).sum()  # Duration as the sum of Higher_Low over 7 days
btc_data['Breakout_Strength'] = ((btc_data['High'] - btc_data['Resistance_Level']) * btc_data['Volume']).rolling(window=7).mean()  # Volume-adjusted breakout strength

In [233]:
#ADX Pattern (Show the strength of the trend)
!pip install ta
import ta
# Calculate ADX, +DI, and -DI using the ta library
btc_data['ADX'] = ta.trend.adx(btc_data['High'], btc_data['Low'], btc_data['Price'], window=14)
btc_data['+DI'] = ta.trend.adx_pos(btc_data['High'], btc_data['Low'], btc_data['Price'], window=14)
btc_data['-DI'] = ta.trend.adx_neg(btc_data['High'], btc_data['Low'], btc_data['Price'], window=14)

# Define the ADX threshold for a strong trend
adx_threshold = 25

# Generate trade signals
btc_data['ADX_Buy_Signal'] = ((btc_data['ADX'] > adx_threshold) & (btc_data['+DI'] > btc_data['-DI'])).astype(int)
btc_data['ADX_Sell_Signal'] = ((btc_data['ADX'] > adx_threshold) & (btc_data['-DI'] > btc_data['+DI'])).astype(int)

In [234]:
# Count the number of breakout signals
breakout_count = btc_data['ADX_Buy_Signal'].sum()
print(f"Number of breakout signals: {breakout_count}")

Number of breakout signals: 1377


In [235]:
# Add continuous features
# ADX_Strength: How far ADX is above the threshold (ADX - 25, 0 if below)
btc_data['ADX_Strength'] = (btc_data['ADX'] - adx_threshold).clip(lower=0)

# ADX_Trend_Duration: Number of consecutive days ADX > 25
btc_data['ADX_Trend_Duration'] = (btc_data['ADX'] > adx_threshold).astype(int).groupby((btc_data['ADX'] <= adx_threshold).astype(int).cumsum()).cumsum()

# ADX_Trend_Direction: +DI minus -DI (positive for uptrend, negative for downtrend)
btc_data['ADX_Trend_Direction'] = btc_data['+DI'] - btc_data['-DI']

In [236]:
# Calculate Stochastic Oscillator (%K and %D)
btc_data['Stochastic_%K'] = ta.momentum.stoch(
    btc_data['High'], btc_data['Low'], btc_data['Price'], window=14, smooth_window=3
)
btc_data['Stochastic_%D'] = ta.momentum.stoch_signal(
    btc_data['High'], btc_data['Low'], btc_data['Price'], window=14, smooth_window=3
)

# Define Stochastic Buy and Sell Signals
btc_data['Stochastic_Buy_Signal'] = np.where(
    (btc_data['Stochastic_%K'] < 20) & (btc_data['Stochastic_%K'] > btc_data['Stochastic_%D']),
    1,  # Buy Signal
    0   # No Signal
)

btc_data['Stochastic_Sell_Signal'] = np.where(
    (btc_data['Stochastic_%K'] > 80) & (btc_data['Stochastic_%K'] < btc_data['Stochastic_%D']),
    1,  # Sell Signal
    0   # No Signal
)

In [237]:
#Bullish and Bearish Engulfing
def detect_bullish_engulfing(data):
    return (
        (data['Price'] > data['Open']) &  # Current candle is bullish
        (data['Price'].shift(1) < data['Open'].shift(1)) &  # Previous candle is bearish
        (data['Open'] < data['Price'].shift(1)) &  # Current open is below previous close
        (data['Price'] > data['Open'].shift(1))    # Current close is above previous open
    )

def detect_bearish_engulfing(data):
    return (
        (data['Price'] < data['Open']) &  # Current candle is bearish
        (data['Price'].shift(1) > data['Open'].shift(1)) &  # Previous candle is bullish
        (data['Open'] > data['Price'].shift(1)) &  # Current open is above previous close
        (data['Price'] < data['Open'].shift(1))    # Current close is below previous open
    )

# Apply the detection functions to btc_data
btc_data['Bullish_Engulfing'] = detect_bullish_engulfing(btc_data).astype(int)
btc_data['Bearish_Engulfing'] = detect_bearish_engulfing(btc_data).astype(int)

# Display the rows where patterns are detected
bullish_patterns = btc_data[btc_data['Bullish_Engulfing'] == 1]
bearish_patterns = btc_data[btc_data['Bearish_Engulfing'] == 1]

In [238]:
breakout_count = btc_data['Bullish_Engulfing'].sum()
print(f"Number of breakout signals: {breakout_count}")

Number of breakout signals: 145


In [239]:

breakout_count = btc_data['Bearish_Engulfing'].sum()
print(f"Number of breakout signals: {breakout_count}")

Number of breakout signals: 130


In [240]:
def detect_hammer(data):
    """
    Detects hammer candlestick patterns.
    A hammer has:
    1. A small body at the top
    2. A long lower shadow (at least 2x the body)
    3. Little or no upper shadow
    """

    # Calculate body and shadows
    data['Body'] = data['Price'] - data['Open']
    data['Upper_Shadow'] = data['High'] - data[['Open', 'Price']].max(axis=1)
    data['Lower_Shadow'] = data[['Open', 'Price']].min(axis=1) - data['Low']

    # Define hammer criteria
    data['Hammer'] = (
        # Body is small (using 0.3 times high-low range as threshold)
        (abs(data['Body']) <= 0.3 * (data['High'] - data['Low'])) &
        # Lower shadow is at least 2x the body length
        (data['Lower_Shadow'] >= 2 * abs(data['Body'])) &
        # Upper shadow is relatively small
        (data['Upper_Shadow'] <= 0.1 * (data['High'] - data['Low']))
    ).astype(int)

    # Clean up temporary columns
    data.drop(['Body', 'Upper_Shadow', 'Lower_Shadow'], axis=1, inplace=True)

    # Count consecutive hammers
    data['Consecutive_Hammer'] = (
        data['Hammer']
        .groupby((data['Hammer'] == 0).cumsum())
        .cumcount()
    )
    data.loc[data['Hammer'] == 0, 'Consecutive_Hammer'] = 0

    return data

# Apply hammer detection to the data
btc_data = detect_hammer(btc_data)

In [241]:
hammer_count = btc_data['Hammer'].sum()
print(f"Total number of hammer patterns detected: {hammer_count}")

# Also get the number of cases where we had consecutive hammers
consecutive_hammers = btc_data[btc_data['Consecutive_Hammer'] > 0]
print(f"Number of cases with consecutive hammers: {len(consecutive_hammers)}")
print(f"Maximum number of consecutive hammers: {btc_data['Consecutive_Hammer'].max()}")

Total number of hammer patterns detected: 94
Number of cases with consecutive hammers: 94
Maximum number of consecutive hammers: 2


In [242]:
def detect_three_white_soldiers(data, min_body_ratio=0.5):
    # Create candle body and shadow lengths
    data['Body'] = data['Price'] - data['Open']
    data['Upper_Shadow'] = data['High'] - data[['Price', 'Open']].max(axis=1)
    data['Lower_Shadow'] = data[['Price', 'Open']].min(axis=1) - data['Low']

    # Initialize Three White Soldiers pattern array
    three_white_soldiers = np.zeros(len(data))

    for i in range(2, len(data)):
        # Check if we have 3 consecutive bullish candles
        bullish_candles = (data['Body'].iloc[i] > 0 and
                          data['Body'].iloc[i-1] > 0 and
                          data['Body'].iloc[i-2] > 0)

        if bullish_candles:
            # Calculate total candle lengths
            candle_lengths = [
                data['High'].iloc[i] - data['Low'].iloc[i],
                data['High'].iloc[i-1] - data['Low'].iloc[i-1],
                data['High'].iloc[i-2] - data['Low'].iloc[i-2]
            ]

            # Calculate body ratios
            body_ratios = [
                data['Body'].iloc[i] / candle_lengths[0],
                data['Body'].iloc[i-1] / candle_lengths[1],
                data['Body'].iloc[i-2] / candle_lengths[2]
            ]

            # Check if each candle has a sufficient body ratio
            strong_bodies = all(ratio >= min_body_ratio for ratio in body_ratios)

            # Check if each candle opens within previous candle's body
            progressive_opens = (
                data['Open'].iloc[i] > data['Open'].iloc[i-1] and
                data['Open'].iloc[i-1] > data['Open'].iloc[i-2]
            )

            # Check if each candle closes higher than previous candle
            progressive_closes = (
                data['Price'].iloc[i] > data['Price'].iloc[i-1] and
                data['Price'].iloc[i-1] > data['Price'].iloc[i-2]
            )

            # If all conditions are met, mark as Three White Soldiers
            if strong_bodies and progressive_opens and progressive_closes:
                three_white_soldiers[i] = 1

    # Add the pattern to the dataframe
    data['Three_White_Soldiers'] = three_white_soldiers

    # Count the occurrences
    pattern_count = int(three_white_soldiers.sum())

    return pattern_count, data['Three_White_Soldiers']

# Apply the detection
pattern_count, three_white_soldiers = detect_three_white_soldiers(btc_data)
print(f"Number of Three White Soldiers patterns detected: {pattern_count}")

Number of Three White Soldiers patterns detected: 50


In [243]:
def detect_three_black_crows(data, min_body_ratio=0.5):
    # Create candle body and shadow lengths
    data['Body'] = data['Price'] - data['Open']
    data['Upper_Shadow'] = data['High'] - data[['Price', 'Open']].max(axis=1)
    data['Lower_Shadow'] = data[['Price', 'Open']].min(axis=1) - data['Low']

    # Initialize Three Black Crows pattern array
    three_black_crows = np.zeros(len(data))

    for i in range(2, len(data)):
        # Check if we have 3 consecutive bearish candles
        bearish_candles = (data['Body'].iloc[i] < 0 and
                          data['Body'].iloc[i-1] < 0 and
                          data['Body'].iloc[i-2] < 0)

        if bearish_candles:
            # Calculate total candle lengths
            candle_lengths = [
                data['High'].iloc[i] - data['Low'].iloc[i],
                data['High'].iloc[i-1] - data['Low'].iloc[i-1],
                data['High'].iloc[i-2] - data['Low'].iloc[i-2]
            ]

            # Calculate body ratios
            body_ratios = [
                abs(data['Body'].iloc[i]) / candle_lengths[0],
                abs(data['Body'].iloc[i-1]) / candle_lengths[1],
                abs(data['Body'].iloc[i-2]) / candle_lengths[2]
            ]

            # Check if each candle has a sufficient body ratio
            strong_bodies = all(ratio >= min_body_ratio for ratio in body_ratios)

            # Check if each candle opens within previous candle's body
            progressive_opens = (
                data['Open'].iloc[i] < data['Open'].iloc[i-1] and
                data['Open'].iloc[i-1] < data['Open'].iloc[i-2]
            )

            # Check if each candle closes lower than previous candle
            progressive_closes = (
                data['Price'].iloc[i] < data['Price'].iloc[i-1] and
                data['Price'].iloc[i-1] < data['Price'].iloc[i-2]
            )

            # If all conditions are met, mark as Three Black Crows
            if strong_bodies and progressive_opens and progressive_closes:
                three_black_crows[i] = 1

    # Add the pattern to the dataframe
    data['Three_Black_Crows'] = three_black_crows

    # Count the occurrences
    pattern_count = int(three_black_crows.sum())

    return pattern_count, data['Three_Black_Crows']

# Apply the detection
pattern_count, three_black_crows = detect_three_black_crows(btc_data)
print(f"Number of Three Black Crows patterns detected: {pattern_count}")

Number of Three Black Crows patterns detected: 28


##Join External Data Source

In [244]:
#Left join gold_price_data, risk_free_data, spy500 to btc_data

btc_data = pd.merge(btc_data, gold_price_data, on='Date', how='left') #till 2025-02-03
btc_data = pd.merge(btc_data, risk_free_data, on='Date', how='left')
btc_data = pd.merge(btc_data, spy_data, on='Date', how='left') #till 2025-04-11

#As other external data have market closeure, all na filled by previous day close data

In [245]:
btc_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4113 entries, 0 to 4112
Data columns (total 100 columns):
 #   Column                                   Non-Null Count  Dtype         
---  ------                                   --------------  -----         
 0   Date                                     4113 non-null   datetime64[ns]
 1   Price                                    4113 non-null   float64       
 2   Open                                     4113 non-null   float64       
 3   High                                     4113 non-null   float64       
 4   Low                                      4113 non-null   float64       
 5   Change                                   4113 non-null   float64       
 6   Volume                                   4113 non-null   float64       
 7   Day_of_Week                              4113 non-null   int32         
 8   Week_of_Year                             4113 non-null   UInt32        
 9   Month                                   

## **Re-scale and Transformation**

**Yeo-Johnson Transformation** for Price, Change, Pct_Change

In [246]:
# For Price, Change, 7D_Pct_Change, 2D_Pct_Change (use yeo-johnson)
from sklearn.preprocessing import PowerTransformer
import pandas as pd

# Initialize PowerTransformer for each column
power_transformer_price = PowerTransformer(method='yeo-johnson', standardize=False)  # Disable standardization
power_transformer_change = PowerTransformer(method='yeo-johnson', standardize=False)
power_transformer_7d_pct_change = PowerTransformer(method='yeo-johnson', standardize=False)
power_transformer_2d_pct_change = PowerTransformer(method='yeo-johnson', standardize=False)

# Apply the Yeo-Johnson transformation to each column
btc_data['YeoJohnson_Price'] = power_transformer_price.fit_transform(btc_data[['Price']])
btc_data['YeoJohnson_Change'] = power_transformer_change.fit_transform(btc_data[['Change']])
btc_data['YeoJohnson_7D_Pct_Change'] = power_transformer_7d_pct_change.fit_transform(btc_data[['7D_Pct_Change']])
btc_data['YeoJohnson_2D_Pct_Change'] = power_transformer_2d_pct_change.fit_transform(btc_data[['2D_Pct_Change']])
btc_data['YeoJohnson_14D_Pct_Change'] = power_transformer_7d_pct_change.fit_transform(btc_data[['14D_Pct_Change']])
btc_data['YeoJohnson_30D_Pct_Change'] = power_transformer_2d_pct_change.fit_transform(btc_data[['30D_Pct_Change']])

# Store lambda values for each column
lambdas = {
    'Price_lambda': power_transformer_price.lambdas_[0],
    'Change_lambda': power_transformer_change.lambdas_[0],
    '7D_Pct_Change_lambda': power_transformer_7d_pct_change.lambdas_[0],
    '2D_Pct_Change_lambda': power_transformer_2d_pct_change.lambdas_[0]
}

# Calculate mean and standard deviation for each transformed column
transformed_means = {
    'Price_mean': btc_data['YeoJohnson_Price'].mean(),
    'Change_mean': btc_data['YeoJohnson_Change'].mean(),
    '7D_Pct_Change_mean': btc_data['YeoJohnson_7D_Pct_Change'].mean(),
    '2D_Pct_Change_mean': btc_data['YeoJohnson_2D_Pct_Change'].mean()
}

transformed_stds = {
    'Price_std': btc_data['YeoJohnson_Price'].std(),
    'Change_std': btc_data['YeoJohnson_Change'].std(),
    '7D_Pct_Change_std': btc_data['YeoJohnson_7D_Pct_Change'].std(),
    '2D_Pct_Change_std': btc_data['YeoJohnson_2D_Pct_Change'].std()
}

# Print the collected lambdas, means, and standard deviations
print(f"Lambdas: {lambdas}")
print(f"Means: {transformed_means}")
print(f"Standard Deviations: {transformed_stds}")

### Step 2: Back-Transformation ###
# Recreate PowerTransformers with stored lambdas for back-transformation
power_transformer_price.lambdas_ = [lambdas['Price_lambda']]
power_transformer_change.lambdas_ = [lambdas['Change_lambda']]
power_transformer_7d_pct_change.lambdas_ = [lambdas['7D_Pct_Change_lambda']]
power_transformer_2d_pct_change.lambdas_ = [lambdas['2D_Pct_Change_lambda']]

Lambdas: {'Price_lambda': 0.1400949314417775, 'Change_lambda': 0.8205213842968994, '7D_Pct_Change_lambda': 0.8756536404438653, '2D_Pct_Change_lambda': 0.892240501612388}
Means: {'Price_mean': 17.920840967250786, 'Change_mean': -0.31190242671149115, '7D_Pct_Change_mean': -0.5123596377593697, '2D_Pct_Change_mean': -0.3859206443970344}
Standard Deviations: {'Price_std': 6.274424897270232, 'Change_std': 5.105807564363485, '7D_Pct_Change_std': 12.486660956829667, '2D_Pct_Change_std': 7.114988997205028}


In [247]:
numerical_variable = btc_data.select_dtypes(include=['number'])
btc_data.columns

Index(['Date', 'Price', 'Open', 'High', 'Low', 'Change', 'Volume',
       'Day_of_Week', 'Week_of_Year', 'Month',
       ...
       'US_10Y', 'Yield_Spread', 'SPY_Price', 'SPY_Volume', 'YeoJohnson_Price',
       'YeoJohnson_Change', 'YeoJohnson_7D_Pct_Change',
       'YeoJohnson_2D_Pct_Change', 'YeoJohnson_14D_Pct_Change',
       'YeoJohnson_30D_Pct_Change'],
      dtype='object', length=106)

**Standardize all numeric variable**

store to scaled_btc_data

In [248]:
#Standardize all numeric variable
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Initialize the MinMaxScaler and StandardScaler
min_max_scaler = MinMaxScaler()
standard_scaler = StandardScaler()

# Apply log transformation to 'Volume' to reduce the range
btc_data['Volume'] = np.log1p(btc_data['Volume'])  # log1p handles log(0) safely

# Apply Scaler
btc_data['Volume'] = min_max_scaler.fit_transform(btc_data[['Volume']])
btc_data['Volume'] = standard_scaler.fit_transform(btc_data[['Volume']])

# List of features for scaling
numeric_features = [
  'Price', 'Open', 'High', 'Low', 'Change', 'RSI_6', 'RSI_12', 'EMA_14',
  'SMA_14', 'OBV', 'MACD', 'MACD_Signal','MACD_above_signal', 'MACD_below_signal','MACD_cum_buy', 'MACD_cum_sell',
  'BB_lower', 'BB_upper','Price_below_BB_lower', 'Price_above_BB_upper', 'BB_cum_Buy', 'BB_cum_Sell',
  'ATR', 'Stop_Loss_Long', 'Stop_Loss_Short', 'VWAP','Support', 'Resistance', 'YeoJohnson_Price',
  'YeoJohnson_Change','YeoJohnson_7D_Pct_Change', 'YeoJohnson_2D_Pct_Change', 'Gold_Price', 'Gold_Volume',
  'US_10Y', 'Yield_Spread','Triangle_Completion', 'Triangle_Height', 'Breakout_Strength',
  'ADX_Strength', 'ATR_Ratio', 'Price_vs_MA50', 'Price_vs_MA200', 'MA_Distance_Ratio', 'OBV_Slope',
  'Stochastic_%K', 'Stochastic_%D', 'SPY_Price', 'SPY_Volume', 'YeoJohnson_14D_Pct_Change', 'YeoJohnson_30D_Pct_Change']

# Fit and transform only the numeric features
scaled_features = standard_scaler.fit_transform(btc_data[numeric_features])

# Create a new DataFrame for scaled data
scaled_btc_data = btc_data.copy()  # Copy original DataFrame

# Replace numeric columns in the new DataFrame with scaled values
scaled_btc_data[numeric_features] = scaled_features

scaled_btc_data.tail()

,Date,Price,Open,High,Low,Change,Volume,Day_of_Week,Week_of_Year,Month,Quarter,Year,RSI_6,RSI_12,EMA_14,SMA_14,OBV,MACD,MACD_Signal,MACD_buy,MACD_sell,MACD_above_signal,MACD_below_signal,MACD_cum_buy,MACD_cum_sell,BB_upper,BB_lower,BB_Buy,BB_Sell,Price_below_BB_lower,Price_above_BB_upper,BB_cum_Buy,BB_cum_Sell,ATR,Stop_Loss_Long,Stop_Loss_Short,VWAP,VWAP_Buy,VWAP_Sell,Support,Resistance,RSI_12_Oversold,Consecutive_RSI_12_Oversold,RSI_12_Overbought,RSI_Divergence,Consecutive_RSI_12_Overbought,ATR_Ratio,MA50,MA200,Price_vs_MA50,Price_vs_MA200,MA_Distance_Ratio,Avg_Volume_20,Volume_Ratio,OBV_Slope,7D_Pct_Change,2D_Pct_Change,14D_Pct_Change,30D_Pct_Change,7D_Positive,2D_Positive,Resistance_Level,Low_Shifted,Higher_Low,Breakout,Ascending_Triangle_Breakout,Avg_Volume,Breakout_With_Volume,Ascending_Triangle_Breakout_With_Volume,Triangle_Completion,Triangle_Height,Triangle_Duration,Breakout_Strength,ADX,+DI,-DI,ADX_Buy_Signal,ADX_Sell_Signal,ADX_Strength,ADX_Trend_Duration,ADX_Trend_Direction,Stochastic_%K,Stochastic_%D,Stochastic_Buy_Signal,Stochastic_Sell_Signal,Bullish_Engulfing,Bearish_Engulfing,Hammer,Consecutive_Hammer,Body,Upper_Shadow,Lower_Shadow,Three_White_Soldiers,Three_Black_Crows,Gold_Price,Gold_Volume,US_10Y,Yield_Spread,SPY_Price,SPY_Volume,YeoJohnson_Price,YeoJohnson_Change,YeoJohnson_7D_Pct_Change,YeoJohnson_2D_Pct_Change,YeoJohnson_14D_Pct_Change,YeoJohnson_30D_Pct_Change
4108,2025-04-01,2.6681,2.5636,2.6108,2.6382,0.4267,-0.3197,1,14,4,2,2025,0.1072,-0.2050,2.6680,2.6849,-1.6037,-0.9833,-1.1640,0,0,0.9649,-0.9495,0.9436,-0.6349,2.5080,2.8489,0,0,-0.2302,-0.3016,-0.1893,-0.2437,1.9580,2.7106,2.6206,2.6392,1,0,2.8768,2.5090,0,0,0,1,0,-0.3484,88264.6580,86012.8685,-0.4248,-0.4184,-0.0077,59.6145,1.1192,0.0088,NaN,-2.3540,NaN,NaN,0,0,85430.2187,81300.1000,1,0,0,58.1386,1,1,-2.2171,-0.4573,1.0000,0.0232,21.9428,17.5662,21.1814,0,0,-0.7804,0,-3.6153,-0.0937,-0.9619,0,0,0,0,0,0,2615.6000,342.0000,112.8000,0.0000,0.0000,NaN,NaN,1.5819,-0.4077,2.0446,-0.7874,1.5873,0.5928,NaN,-0.3182,NaN,NaN
4109,2025-04-02,2.5599,2.6710,2.7261,2.6349,-0.4898,-0.1801,2,14,4,2,2025,-0.6900,-0.6509,2.6565,2.6722,-1.6037,-1.0604,-1.1540,0,0,0.9649,-0.9495,1.0316,-0.6349,2.5026,2.8618,0,0,-0.2302,-0.3016,-0.1893,-0.2437,2.1336,2.5743,2.5364,2.6414,0,1,2.9173,2.5090,0,0,0,0,0,-0.2541,87999.7240,86122.9355,-0.5880,-0.4931,-0.0233,60.8545,1.7098,0.0088,NaN,1.6258,NaN,NaN,0,1,86166.9891,82435.8000,0,0,0,64.1400,1,1,-2.2171,-0.3053,1.0000,0.0232,20.8923,21.3839,18.4984,0,0,-0.7804,0,2.8855,-1.3276,-0.9544,0,0,0,1,0,0,-2641.0000,3211.3000,166.1000,0.0000,0.0000,NaN,NaN,1.6109,-0.4205,2.0730,-0.3288,1.5628,-0.6500,NaN,0.2633,NaN,NaN
4110,2025-04-03,2.5859,2.5625,2.5466,2.5911,0.0759,-0.2506,3,14,4,2,2025,-0.4568,-0.5222,2.6500,2.6691,-1.6037,-1.0656,-1.1470,0,0,0.9649,-0.9495,1.1197,-0.6349,2.5028,2.8579,0,0,-0.2302,-0.3016,-0.1893,-0.2437,2.0840,2.6078,2.5560,2.5746,1,0,2.9329,2.5090,0,0,0,0,0,-0.2800,87705.2860,86238.7555,-0.5254,-0.4789,-0.0448,60.9505,1.3642,0.0088,NaN,0.4722,NaN,NaN,0,1,85602.4668,82358.9000,0,0,0,67.9486,0,0,-2.2171,-0.2738,1.0000,0.0232,19.4873,20.1959,19.7080,0,0,-0.7804,0,0.4880,-1.0310,-0.8706,0,0,0,0,0,0,636.8000,749.5000,1208.8000,0.0000,0.0000,NaN,NaN,1.4757,-0.3434,1.8502,0.7420,1.5687,0.2038,NaN,0.1185,NaN,NaN
4111,2025-04-04,2.6149,2.5887,2.5785,2.6069,0.0876,-0.2204,4,14,4,2,2025,-0.1947,-0.3769,2.6483,2.6685,-1.6037,-1.0092,-1.1297,0,0,0.9649,-0.9495,1.2077,-0.6349,2.5023,2.8563,0,0,-0.2302,-0.3016,-0.1893,-0.2437,2.0638,2.6415,2.5809,2.6003,0,0,2.9329,2.5090,0,0,0,1,0,-0.2959,87450.4160,86362.3965,-0.4601,-0.4630,-0.0645,63.6075,1.4391,0.0088,NaN,NaN,NaN,NaN,0,0,85377.8751,81313.8000,1,0,0,68.8471,0,0,-1.4982,-0.3088,2.0000,0.0232,18.4926,20.5984,18.4280,0,0,-0.7804,0,2.1704,-0.7003,-1.0859,0,0,0,0,0,0,706.5000,837.4000,1469.1000,0.0000,0.0000,NaN,NaN,1.4274,-0.3691,1.5985,2.7129,1.5753,0.2178,NaN,NaN,NaN,NaN
4112,2025-04-05,2.6020,2.6181,2.5603,2.6360,-0.0922,-0.5414,5,14,4,2,2025,-0.3122,-0.4365,2.6451,2.6677,-1.6

In [249]:
scaled_btc_data.describe().T

,count,mean,min,25%,50%,75%,max,std
Date,4113,2019-08-19 00:00:00,2014-01-01 00:00:00,2016-10-25 00:00:00,2019-08-19 00:00:00,2022-06-12 00:00:00,2025-04-05 00:00:00,NaN
Price,4113.0000,-0.0000,-0.8182,-0.7917,-0.4577,0.4413,3.5286,1.0001
Open,4113.0000,0.0000,-0.8180,-0.7914,-0.4573,0.4396,3.5320,1.0001
High,4113.0000,-0.0000,-0.8175,-0.7930,-0.4556,0.4520,3.5637,1.0001
Low,4113.0000,0.0000,-0.8171,-0.7899,-0.4567,0.4419,3.5997,1.0001
...,...,...,...,...,...,...,...,...
YeoJohnson_Change,4113.0000,-0.0000,-19.8210,-0.2141,0.0786,0.3600,28.1859,1.0001
YeoJohnson_7D_Pct_Change,4106.0000,-0.0000,-10.0222,-0.3169,0.0880,0.4238,16.4748,1.0001
YeoJohnson_2D_Pct_Change,4111.0000,-0.0000,-14.6786,-0.2523,0.0879,0.3790,22.1681,1.0001
YeoJohnson_14D_Pct_Change,4099.0000,0.0000,-8.0804,-0.3911,0.0681,0.4544,15.1742,1.0001


# **Please use scaled_btc_data for analysis**

- missing values
- rolling window counts - how many buy signals in the last 7 days
- Combine Indicators

**Drop NA before Analysis**

In [250]:
scaled_btc_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4113 entries, 0 to 4112
Columns: 106 entries, Date to YeoJohnson_30D_Pct_Change
dtypes: UInt32(1), datetime64[ns](1), float64(73), int32(4), int64(27)
memory usage: 3.3 MB


In [251]:
# Drop rows with NaN values in scaled_btc_data
scaled_btc_data = scaled_btc_data.dropna()

# Optionally, reset the index after dropping rows
scaled_btc_data = scaled_btc_data.reset_index(drop=True)

In [252]:
scaled_btc_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3853 entries, 0 to 3852
Columns: 106 entries, Date to YeoJohnson_30D_Pct_Change
dtypes: UInt32(1), datetime64[ns](1), float64(73), int32(4), int64(27)
memory usage: 3.0 MB


In [253]:
#Check data are consecutive
# Ensure 'Date' is sorted
scaled_btc_data = scaled_btc_data.sort_values('Date')

# Calculate the difference between consecutive dates
scaled_btc_data['Date_Diff'] = scaled_btc_data['Date'].diff().dt.days

# Check if all differences are 1 day
if (scaled_btc_data['Date_Diff'].iloc[1:] == 1).all():
    print("All dates are consecutive.")
else:
    print("There are missing dates.")
    print(scaled_btc_data[scaled_btc_data['Date_Diff'] > 1])  # Display rows with missing dates

# Drop the 'Date_Diff' column after the check
scaled_btc_data.drop(columns=['Date_Diff'], inplace=True)

All dates are consecutive.


In [254]:
# Check the start and end dates
start_date = scaled_btc_data['Date'].min()
end_date = scaled_btc_data['Date'].max()

print(f"Data starts from: {start_date}")
print(f"Data ends at: {end_date}")

Data starts from: 2014-07-19 00:00:00
Data ends at: 2025-02-03 00:00:00


#Exploratory Data Analysis

In [255]:
#Print the name of the columns
print(scaled_btc_data.columns)

Index(['Date', 'Price', 'Open', 'High', 'Low', 'Change', 'Volume',
       'Day_of_Week', 'Week_of_Year', 'Month',
       ...
       'US_10Y', 'Yield_Spread', 'SPY_Price', 'SPY_Volume', 'YeoJohnson_Price',
       'YeoJohnson_Change', 'YeoJohnson_7D_Pct_Change',
       'YeoJohnson_2D_Pct_Change', 'YeoJohnson_14D_Pct_Change',
       'YeoJohnson_30D_Pct_Change'],
      dtype='object', length=106)


In [256]:
# Define target, 7D_Pct_Change or YeoJohnson_7D_Pct_Change or 2D_Pct_Change or YeoJohnson_2D_Pct_Change
target = '7D_Pct_Change'

In [257]:
scaled_btc_data

,Date,Price,Open,High,Low,Change,Volume,Day_of_Week,Week_of_Year,Month,Quarter,Year,RSI_6,RSI_12,EMA_14,SMA_14,OBV,MACD,MACD_Signal,MACD_buy,MACD_sell,MACD_above_signal,MACD_below_signal,MACD_cum_buy,MACD_cum_sell,BB_upper,BB_lower,BB_Buy,BB_Sell,Price_below_BB_lower,Price_above_BB_upper,BB_cum_Buy,BB_cum_Sell,ATR,Stop_Loss_Long,Stop_Loss_Short,VWAP,VWAP_Buy,VWAP_Sell,Support,Resistance,RSI_12_Oversold,Consecutive_RSI_12_Oversold,RSI_12_Overbought,RSI_Divergence,Consecutive_RSI_12_Overbought,ATR_Ratio,MA50,MA200,Price_vs_MA50,Price_vs_MA200,MA_Distance_Ratio,Avg_Volume_20,Volume_Ratio,OBV_Slope,7D_Pct_Change,2D_Pct_Change,14D_Pct_Change,30D_Pct_Change,7D_Positive,2D_Positive,Resistance_Level,Low_Shifted,Higher_Low,Breakout,Ascending_Triangle_Breakout,Avg_Volume,Breakout_With_Volume,Ascending_Triangle_Breakout_With_Volume,Triangle_Completion,Triangle_Height,Triangle_Duration,Breakout_Strength,ADX,+DI,-DI,ADX_Buy_Signal,ADX_Sell_Signal,ADX_Strength,ADX_Trend_Duration,ADX_Trend_Direction,Stochastic_%K,Stochastic_%D,Stochastic_Buy_Signal,Stochastic_Sell_Signal,Bullish_Engulfing,Bearish_Engulfing,Hammer,Consecutive_Hammer,Body,Upper_Shadow,Lower_Shadow,Three_White_Soldiers,Three_Black_Crows,Gold_Price,Gold_Volume,US_10Y,Yield_Spread,SPY_Price,SPY_Volume,YeoJohnson_Price,YeoJohnson_Change,YeoJohnson_7D_Pct_Change,YeoJohnson_2D_Pct_Change,YeoJohnson_14D_Pct_Change,YeoJohnson_30D_Pct_Change
0,2014-07-19,-0.7970,-0.7968,-0.7985,-0.7947,-0.0469,-1.1506,5,29,7,3,2014,0.1366,0.0985,-0.8001,-0.7993,0.5742,-0.1236,-0.1298,0,0,-1.0364,1.0532,-0.6408,0.1739,-0.8114,-0.7822,0,0,-0.2302,-0.3016,-0.1893,-0.2437,-0.8043,-0.7909,-0.8051,-0.7969,0,1,-0.7857,-0.8119,0,0,0,1,0,-0.6627,620.8300,603.9725,-0.1534,-0.3060,-0.0058,3.3210,1.1924,0.0088,-4.9442,-0.3030,-5.6300,-26.2839,0,0,630.6235,617.9000,1,0,0,3.0243,1,1,-0.0602,-0.7577,4.0000,0.0232,8.2364,22.9691,20.4122,0,0,-0.7804,0,2.5568,0.2890,0.2328,0,0,0,0,0,0,-0.5000,6.6000,1.9000,0.0000,0.0000,-0.6813,-0.9412,-0.0306,1.7637,-1.1356,0.7065,-1.1886,0.0473,-0.4297,0.0107,-0.3988,-1.5678
1,2014-07-20,-0.7973,-0.7968,-0.7987,-0.7949,-0.1784,-1.2801,6,29,7,3,2014,-0.3852,-0.1610,-0.8001,-0.7993,0.5742,-0.1242,-0.1301,0,0,-1.0364,1.0532,-0.6408,0.2895,-0.8114,-0.7823,0,0,-0.2302,-0.3016,-0.1893,-0.2437,-0.8048,-0.7912,-0.8053,-0.7971,0,0,-0.7853,-0.8119,0,0,0,0,0,-0.6801,620.6920,602.9980,-0.2102,-0.3254,0.0009,3.2445,0.7089,0.0088,-4.1868,0.2415,-5.1691,-21.8841,0,1,629.9426,625.1000,0,0,0,3.1100,0,0,-0.7792,-0.7743,3.0000,0.0232,7.6616,22.1225,22.0392,0,0,-0.7804,0,0.0833,-0.3778,0.0905,0,0,0,0,0,0,-6.0000,0.9000,1.0000,0.0000,0.0000,-0.6813,-0.9412,-0.0306,1.7637,-1.1356,0.7065,-1.1924,-0.1423,-0.3513,0.0876,-0.3640,-1.2962
2,2014-07-21,-0.7971,-0.7971,-0.7988,-0.7952,0.0613,-1.2587,0,30,7,3,2014,-0.0211,0.0062,-0.8001,-0.7993,0.5742,-0.1243,-0.1304,0,0,-1.0364,1.0532,-0.6408,0.4050,-0.8115,-0.7822,0,0,-0.2302,-0.3016,-0.1893,-0.2437,-0.8050,-0.7909,-0.8052,-0.7972,1,0,-0.7853,-0.8119,0,0,0,0,0,-0.6974,620.1980,601.8390,-0.1656,-0.3048,0.0046,3.1655,0.7992,0.0088,-5.9510,-0.5439,-5.1672,-17.3252,0,0,628.9320,620.0000,0,0,0,3.1071,0,0,-0.7792,-0.7859,3.0000,0.0232,7.4860,20.9739,23.2765,0,0,-0.7804,0,-2.3026,-0.0245,-0.0405,0,0,0,0,0,0,4.1000,0.8000,6.0000,0.0000,0.0000,-0.6767,-1.1973,-0.0402,1.7508,-1.1380,-0.5092,-1.1898,0.1860,-0.5361,-0.0251,-0.3639,-1.0208
3,2014-07-22,-0.7972,-0.7969,-0.7987,-0.7949,-0.0980,-1.1975,1,30,7,3,2014,-0.2645,-0.1135,-0.8001,-0.7992,0.5742,-0.1247,-0.1307,0,0,-1.0364,1.0532,-0.6408,0.5206,-0.8117,-0.7821,0,0,-0.2302,-0.3016,-0.1893,-0.2437,-0.8054,-0.7910,-0.8053,-0.7971,0,1,-0.7852,-0.8121,0,0,0,0,0,-0.7132,619.5380,600.5300,-0.1844,-0.3096,0.0095,3.0905,1.0613,0.0088,-5.8956,-3.3092,-5.4137,-16.8032,0,0,628.8990,615.0000,1,0,0,3.0486,0,0,-0.7792,-0.7769,3.0000,0.0232,7.0786,21.4914,22.2712,0,0,-0.7804,0,-0.7798,-0.3800,-0.2779,0,0,0,0,0,0,-2.6000,3.7000,2.5000,0.0000,0.0000,-0.6893,-1.0473,-0.0499,1.7637,-1.1323,-0.5074,-1.1915,-0.0240,-0.5302,-0.4848,-0.3825,-0

In [258]:
# Define predictors, all columns except the 7D_Pct_Change or YeoJohnson_7D_Pct_Change or 2D_Pct_Change or YeoJohnson_2D_Pct_Change
predictors = scaled_btc_data.drop(columns=[target,
                                           "YeoJohnson_7D_Pct_Change", "2D_Pct_Change", "YeoJohnson_2D_Pct_Change",'Price', 'Open', 'High',
                                           'Low', 'Change', 'VWAP', 'VWAP_Buy', 'VWAP_Sell', 'Resistance_Level', 'Low_Shifted', 'Higher_Low',
                                           'Breakout', 'Ascending_Triangle_Breakout', 'Avg_Volume', 'Breakout_With_Volume', 'Avg_Volume_20'
                                           ], axis=1).columns.tolist()
# Moving date to index
scaled_btc_data.set_index('Date', inplace=True)
# Display the first few rows of the DataFrame
scaled_btc_data.head()

,Price,Open,High,Low,Change,Volume,Day_of_Week,Week_of_Year,Month,Quarter,Year,RSI_6,RSI_12,EMA_14,SMA_14,OBV,MACD,MACD_Signal,MACD_buy,MACD_sell,MACD_above_signal,MACD_below_signal,MACD_cum_buy,MACD_cum_sell,BB_upper,BB_lower,BB_Buy,BB_Sell,Price_below_BB_lower,Price_above_BB_upper,BB_cum_Buy,BB_cum_Sell,ATR,Stop_Loss_Long,Stop_Loss_Short,VWAP,VWAP_Buy,VWAP_Sell,Support,Resistance,RSI_12_Oversold,Consecutive_RSI_12_Oversold,RSI_12_Overbought,RSI_Divergence,Consecutive_RSI_12_Overbought,ATR_Ratio,MA50,MA200,Price_vs_MA50,Price_vs_MA200,MA_Distance_Ratio,Avg_Volume_20,Volume_Ratio,OBV_Slope,7D_Pct_Change,2D_Pct_Change,14D_Pct_Change,30D_Pct_Change,7D_Positive,2D_Positive,Resistance_Level,Low_Shifted,Higher_Low,Breakout,Ascending_Triangle_Breakout,Avg_Volume,Breakout_With_Volume,Ascending_Triangle_Breakout_With_Volume,Triangle_Completion,Triangle_Height,Triangle_Duration,Breakout_Strength,ADX,+DI,-DI,ADX_Buy_Signal,ADX_Sell_Signal,ADX_Strength,ADX_Trend_Duration,ADX_Trend_Direction,Stochastic_%K,Stochastic_%D,Stochastic_Buy_Signal,Stochastic_Sell_Signal,Bullish_Engulfing,Bearish_Engulfing,Hammer,Consecutive_Hammer,Body,Upper_Shadow,Lower_Shadow,Three_White_Soldiers,Three_Black_Crows,Gold_Price,Gold_Volume,US_10Y,Yield_Spread,SPY_Price,SPY_Volume,YeoJohnson_Price,YeoJohnson_Change,YeoJohnson_7D_Pct_Change,YeoJohnson_2D_Pct_Change,YeoJohnson_14D_Pct_Change,YeoJohnson_30D_Pct_Change
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2014-07-19,-0.7970,-0.7968,-0.7985,-0.7947,-0.0469,-1.1506,5,29,7,3,2014,0.1366,0.0985,-0.8001,-0.7993,0.5742,-0.1236,-0.1298,0,0,-1.0364,1.0532,-0.6408,0.1739,-0.8114,-0.7822,0,0,-0.2302,-0.3016,-0.1893,-0.2437,-0.8043,-0.7909,-0.8051,-0.7969,0,1,-0.7857,-0.8119,0,0,0,1,0,-0.6627,620.8300,603.9725,-0.1534,-0.3060,-0.0058,3.3210,1.1924,0.0088,-4.9442,-0.3030,-5.6300,-26.2839,0,0,630.6235,617.9000,1,0,0,3.0243,1,1,-0.0602,-0.7577,4.0000,0.0232,8.2364,22.9691,20.4122,0,0,-0.7804,0,2.5568,0.2890,0.2328,0,0,0,0,0,0,-0.5000,6.6000,1.9000,0.0000,0.0000,-0.6813,-0.9412,-0.0306,1.7637,-1.1356,0.7065,-1.1886,0.0473,-0.4297,0.0107,-0.3988,-1.5678
2014-07-20,-0.7973,-0.7968,-0.7987,-0.7949,-0.1784,-1.2801,6,29,7,3,2014,-0.3852,-0.1610,-0.8001,-0.7993,0.5742,-0.1242,-0.1301,0,0,-1.0364,1.0532,-0.6408,0.2895,-0.8114,-0.7823,0,0,-0.2302,-0.3016,-0.1893,-0.2437,-0.8048,-0.7912,-0.8053,-0.7971,0,0,-0.7853,-0.8119,0,0,0,0,0,-0.6801,620.6920,602.9980,-0.2102,-0.3254,0.0009,3.2445,0.7089,0.0088,-4.1868,0.2415,-5.1691,-21.8841,0,1,629.9426,625.1000,0,0,0,3.1100,0,0,-0.7792,-0.7743,3.0000,0.0232,7.6616,22.1225,22.0392,0,0,-0.7804,0,0.0833,-0.3778,0.0905,0,0,0,0,0,0,-6.0000,0.9000,1.0000,0.0000,0.0000,-0.6813,-0.9412,-0.0306,1.7637,-1.1356,0.7065,-1.1924,-0.1423,-0.3513,0.0876,-0.3640,-1.2962
2014-07-21,-0.7971,-0.7971,-0.7988,-0.7952,0.0613,-1.2587,0,30,7,3,2014,-0.0211,0.0062,-0.8001,-0.7993,0.5742,-0.1243,-0.1304,0,0,-1.0364,1.0532,-0.6408,0.4050,-0.8115,-0.7822,0,0,-0.2302,-0.3016,-0.1893,-0.2437,-0.8050,-0.7909,-0.8052,-0.7972,1,0,-0.7853,-0.8119,0,0,0,0,0,-0.6974,620.1980,601.8390,-0.1656,-0.3048,0.0046,3.1655,0.7992,0.0088,-5.9510,-0.5439,-5.1672,-17.3252,0,0,628.9320,620.0000,0,0,0,3.1071,0,0,-0.7792,-0.7859,3.0000,0.0232,7.4860,20.9739,23.2765,0,0,-0.7804,0,-2.3026,-0.0245,-0.0405,0,0,0,0,0,0,4.1000,0.8000,6.0000,0.0000,0.0000,-0.6767,-1.1973,-0.0402,1.7508,-1.1380,-0.5092,-1.1898,0.1860,-0.5361,-0.0251,-0.3639,-1.0208
2014-07-22,-0.7972,-0.7969,-0.7987,-0.7949,-0.0980,-1.1975,1,30,7,3,2014,-0.2645,-0.1135,-0.8001,-0.7992,0.5742,-0.1247,-0.1307,0,0,-1.0364,1.0532,-0.6408,0.5206,-0.8117,-0.7821,0,0,-0.2302,-0.3016,-0.1893,-0.2437,-0.8054,-0.7910,-0.8053,-0.7971,0,1,-0.7852,-0.8121,0,0,0,0,0,-0.7132,619.5380,600.5300,-0.1844,-0.3096,0.0095,3.0905,1.0613,0.0088,-5.8956,-3.3092,-5.4137,-16.8032,0,0,628.8990,615.0000,1,0,0,3.0486,0,0,-0.7792,-0.7769,3.0000,0.0232,7.0786,21.4914,22.2712,0,0,-0.7804,0,-0.7798,-0.3800,-0.2779,0,0,0,0,0,0,-2.6000,3.7000,2.5000,0.0000

In [259]:
# Check the predictors
predictors

['Date',
 'Volume',
 'Day_of_Week',
 'Week_of_Year',
 'Month',
 'Quarter',
 'Year',
 'RSI_6',
 'RSI_12',
 'EMA_14',
 'SMA_14',
 'OBV',
 'MACD',
 'MACD_Signal',
 'MACD_buy',
 'MACD_sell',
 'MACD_above_signal',
 'MACD_below_signal',
 'MACD_cum_buy',
 'MACD_cum_sell',
 'BB_upper',
 'BB_lower',
 'BB_Buy',
 'BB_Sell',
 'Price_below_BB_lower',
 'Price_above_BB_upper',
 'BB_cum_Buy',
 'BB_cum_Sell',
 'ATR',
 'Stop_Loss_Long',
 'Stop_Loss_Short',
 'Support',
 'Resistance',
 'RSI_12_Oversold',
 'Consecutive_RSI_12_Oversold',
 'RSI_12_Overbought',
 'RSI_Divergence',
 'Consecutive_RSI_12_Overbought',
 'ATR_Ratio',
 'MA50',
 'MA200',
 'Price_vs_MA50',
 'Price_vs_MA200',
 'MA_Distance_Ratio',
 'Volume_Ratio',
 'OBV_Slope',
 '14D_Pct_Change',
 '30D_Pct_Change',
 '7D_Positive',
 '2D_Positive',
 'Ascending_Triangle_Breakout_With_Volume',
 'Triangle_Completion',
 'Triangle_Height',
 'Triangle_Duration',
 'Breakout_Strength',
 'ADX',
 '+DI',
 '-DI',
 'ADX_Buy_Signal',
 'ADX_Sell_Signal',
 'ADX_Strength'

##Univariate Analysis

In [260]:
cols = predictors[1:] + [target]
ncols = 5
nrows = math.ceil(len(cols) / ncols)

fig, axes = plt.subplots(nrows, ncols, figsize=(20, 4 * nrows))
axes = axes.flatten()

for ax, col in zip(axes, cols):
    scaled_btc_data[col].hist(
        bins=30,
        ax=ax,
        edgecolor='black'
    )
    ax.set_title(col)
    ax.set_xlabel('')
    ax.set_ylabel('')

# turn off any unused subplots
for ax in axes[len(cols):]:
    fig.delaxes(ax)

plt.tight_layout()
plt.show()

NameError: name 'math' is not defined

##Bivariate Analysis

In [ ]:
# Select numeric columns
numeric_variables = scaled_btc_data.select_dtypes(include=['float64']).columns.tolist()

# Identify 2-dimensional columns
for column in numeric_variables:
    if len(scaled_btc_data[column].shape) != 1:  # Check if the column is not 1D
        print(f"Column '{column}' is 2-dimensional with shape {scaled_btc_data[column].shape}")

# Filter out 2D columns and remove target from predictors
numeric_variables = [col for col in numeric_variables if len(scaled_btc_data[col].shape) == 1 and col != target]

# Drop rows with missing values
data_numeric = scaled_btc_data[numeric_variables + [target]].dropna()

# Define number of rows and columns for the subplot grid
n_cols = 3
n_rows = (len(numeric_variables) + n_cols - 1) // n_cols  # Ceiling division

# Create figure with subplots
fig = plt.figure(figsize=(15, 5 * n_rows))

# Create scatter plots
for i, column in enumerate(numeric_variables, 1):
    ax = plt.subplot(n_rows, n_cols, i)
    sns.scatterplot(data=data_numeric, x=column, y=target, ax=ax)
    ax.set_title(f"{column} vs {target}")
    ax.set_xlabel(column)
    ax.set_ylabel(target)

    # Rotate x-axis labels if they're too long
    plt.xticks(rotation=45)

# Adjust layout to prevent overlap
plt.tight_layout()
plt.show()

In [ ]:
# Create correlation matrix
correlation_matrix = data_numeric.corr()

plt.figure(figsize=(20, 20))

# Create heatmap with annotations
sns.heatmap(correlation_matrix,
            annot=True,
            cmap='coolwarm',
            center=0,
            square=True,
            fmt='.2f',
            linewidths=0.5,
            cbar_kws={"shrink": .5})

plt.title('Correlation Heatmap: Numeric Variables vs Price Change', pad=20)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()

plt.show()

In [ ]:
# Select binary columns
binary_variables = scaled_btc_data.select_dtypes(include=['int64']).columns.tolist()

# Identify 2-dimensional columns
for column in binary_variables:
    if len(scaled_btc_data[column].shape) != 1:  # Check if the column is not 1D
        print(f"Column '{column}' is 2-dimensional with shape {scaled_btc_data[column].shape}")

# Filter out 2D columns and remove target from predictors
binary_variables = [col for col in binary_variables if len(scaled_btc_data[col].shape) == 1 and col != target]

# Drop rows with missing values
data_binary = scaled_btc_data[binary_variables + [target]].dropna()

# Define number of rows and columns for the subplot grid
n_cols = 3
n_rows = (len(binary_variables) + n_cols - 1) // n_cols  # Ceiling division

# Create figure with subplots
fig = plt.figure(figsize=(15, 5 * n_rows))

# Create scatter plots
for i, column in enumerate(binary_variables, 1):
    ax = plt.subplot(n_rows, n_cols, i)
    sns.scatterplot(data=data_binary, x=column, y=target, ax=ax)
    ax.set_title(f"{column} vs {target}")
    ax.set_xlabel(column)
    ax.set_ylabel(target)

    # Rotate x-axis labels if they're too long
    plt.xticks(rotation=45)

# Adjust layout to prevent overlap
plt.tight_layout()
plt.show()

In [ ]:
# Create correlation matrix
correlation_matrix = data_binary.corr()

plt.figure(figsize=(12, 10))

# Create heatmap with annotations
sns.heatmap(correlation_matrix,
            annot=True,
            cmap='coolwarm',
            center=0,
            square=True,
            fmt='.3f',
            linewidths=0.5,
            cbar_kws={"shrink": .5})

plt.title('Correlation Heatmap: Binary Variables vs Price Change', pad=20)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()

plt.show()

# Model Building

- Time-Based Splitting
- Selecting predictors
- Interaction terms

In [263]:
from itertools import chain, combinations

def gen_combinations(input):
    return sum([list(map(list, combinations(input, i))) for i in range(1,len(input) + 1)], [])

def gen_interactions(X,de,pname):
    s = gen_combinations(range(len(pname)))
    VX = []
    VarX = []
    for i in range(len(s)):
        if len(s[i]) <= de:
            VX.append(np.prod(X[:,s[i]],axis=1))
            VarX.append(list([pname[x] for x in s[i]]))
    VarX2 = []
    for x in VarX:
        if len(x) > 1:
            VarX2.append('_'.join(x))
        else:
            VarX2.append(x[0])
    VX = pd.DataFrame(np.array(VX).T,columns=VarX2)
    return VX, VarX2

In [264]:
# Step 1: Define relevant predictors (skip time-based and unscaled columns)
relevant_predictors = [
    'VWAP_Buy', 'MACD_buy', 'BB_Buy', 'RSI_12_Oversold', 'RSI_Divergence',
    'VWAP', 'MACD', 'RSI_6', 'RSI_12', 'ATR'
]

# Step 2: Exclude redundant or invalid combinations
excluded_pairs = [
    ('VWAP_Buy', 'VWAP_Sell'),
    ('MACD_buy', 'MACD_sell'),
    ('BB_Buy', 'BB_Sell')
]

# Step 3: Filter the dataset for only relevant predictors
filtered_data = scaled_btc_data[relevant_predictors].dropna()
X = filtered_data.values  # Extract as numpy array
pname = filtered_data.columns.tolist()  # List of predictor names

# Step 4: Generate interaction terms
interaction_terms, interaction_names = gen_interactions(X, de=2, pname=pname)

# Step 5: Remove excluded interaction terms
valid_interactions = []
valid_names = []

for i, name in enumerate(interaction_names):
    # Split interaction term (e.g., "VWAP_Buy_MACD_buy" -> ["VWAP_Buy", "MACD_buy"])
    components = name.split('_')

    # Check if the pair is in the excluded list
    if len(components) == 2 and tuple(sorted(components)) in excluded_pairs:
        continue  # Skip this interaction term

    # Otherwise, keep it
    valid_interactions.append(interaction_terms.iloc[:, i])
    valid_names.append(name)

# Step 6: Convert valid interactions back to a DataFrame
interaction_df = pd.DataFrame(valid_interactions).T  # Transpose to align rows
interaction_df.columns = valid_names  # Set column names

# Step 7: Append interaction terms to the original dataset
scaled_btc_data_with_interactions = pd.concat([scaled_btc_data, interaction_df], axis=1)

# Display the first few rows of the updated dataset
scaled_btc_data_with_interactions.head()

,Price,Open,High,Low,Change,Volume,Day_of_Week,Week_of_Year,Month,Quarter,Year,RSI_6,RSI_12,EMA_14,SMA_14,OBV,MACD,MACD_Signal,MACD_buy,MACD_sell,MACD_above_signal,MACD_below_signal,MACD_cum_buy,MACD_cum_sell,BB_upper,BB_lower,BB_Buy,BB_Sell,Price_below_BB_lower,Price_above_BB_upper,BB_cum_Buy,BB_cum_Sell,ATR,Stop_Loss_Long,Stop_Loss_Short,VWAP,VWAP_Buy,VWAP_Sell,Support,Resistance,RSI_12_Oversold,Consecutive_RSI_12_Oversold,RSI_12_Overbought,RSI_Divergence,Consecutive_RSI_12_Overbought,ATR_Ratio,MA50,MA200,Price_vs_MA50,Price_vs_MA200,MA_Distance_Ratio,Avg_Volume_20,Volume_Ratio,OBV_Slope,7D_Pct_Change,2D_Pct_Change,14D_Pct_Change,30D_Pct_Change,7D_Positive,2D_Positive,Resistance_Level,Low_Shifted,Higher_Low,Breakout,Ascending_Triangle_Breakout,Avg_Volume,Breakout_With_Volume,Ascending_Triangle_Breakout_With_Volume,Triangle_Completion,Triangle_Height,Triangle_Duration,Breakout_Strength,ADX,+DI,-DI,ADX_Buy_Signal,ADX_Sell_Signal,ADX_Strength,ADX_Trend_Duration,ADX_Trend_Direction,Stochastic_%K,Stochastic_%D,Stochastic_Buy_Signal,Stochastic_Sell_Signal,Bullish_Engulfing,Bearish_Engulfing,Hammer,Consecutive_Hammer,Body,Upper_Shadow,Lower_Shadow,Three_White_Soldiers,Three_Black_Crows,Gold_Price,Gold_Volume,US_10Y,Yield_Spread,SPY_Price,SPY_Volume,YeoJohnson_Price,YeoJohnson_Change,YeoJohnson_7D_Pct_Change,YeoJohnson_2D_Pct_Change,YeoJohnson_14D_Pct_Change,YeoJohnson_30D_Pct_Change,VWAP_Buy,MACD_buy,BB_Buy,RSI_12_Oversold,RSI_Divergence,VWAP,MACD,RSI_6,RSI_12,ATR,VWAP_Buy_MACD_buy,VWAP_Buy_BB_Buy,VWAP_Buy_RSI_12_Oversold,VWAP_Buy_RSI_Divergence,VWAP_Buy_VWAP,VWAP_Buy_MACD,VWAP_Buy_RSI_6,VWAP_Buy_RSI_12,VWAP_Buy_ATR,MACD_buy_BB_Buy,MACD_buy_RSI_12_Oversold,MACD_buy_RSI_Divergence,MACD_buy_VWAP,MACD_buy_MACD,MACD_buy_RSI_6,MACD_buy_RSI_12,MACD_buy_ATR,BB_Buy_RSI_12_Oversold,BB_Buy_RSI_Divergence,BB_Buy_VWAP,BB_Buy_MACD,BB_Buy_RSI_6,BB_Buy_RSI_12,BB_Buy_ATR,RSI_12_Oversold_RSI_Divergence,RSI_12_Oversold_VWAP,RSI_12_Oversold_MACD,RSI_12_Oversold_RSI_6,RSI_12_Oversold_RSI_12,RSI_12_Oversold_ATR,RSI_Divergence_VWAP,RSI_Divergence_MACD,RSI_Divergence_RSI_6,RSI_Divergence_RSI_12,RSI_Divergence_ATR,VWAP_MACD,VWAP_RSI_6,VWAP_RSI_12,VWAP_ATR,MACD_RSI_6,MACD_RSI_12,MACD_ATR,RSI_6_RSI_12,RSI_6_ATR,RSI_12_ATR
2014-07-19 00:00:00,-0.7970,-0.7968,-0.7985,-0.7947,-0.0469,-1.1506,5.0000,29,7.0000,3.0000,2014.0000,0.1366,0.0985,-0.8001,-0.7993,0.5742,-0.1236,-0.1298,0.0000,0.0000,-1.0364,1.0532,-0.6408,0.1739,-0.8114,-0.7822,0.0000,0.0000,-0.2302,-0.3016,-0.1893,-0.2437,-0.8043,-0.7909,-0.8051,-0.7969,0.0000,1.0000,-0.7857,-0.8119,0.0000,0.0000,0.0000,1.0000,0.0000,-0.6627,620.8300,603.9725,-0.1534,-0.3060,-0.0058,3.3210,1.1924,0.0088,-4.9442,-0.3030,-5.6300,-26.2839,0.0000,0.0000,630.6235,617.9000,1.0000,0.0000,0.0000,3.0243,1.0000,1.0000,-0.0602,-0.7577,4.0000,0.0232,8.2364,22.9691,20.4122,0.0000,0.0000,-0.7804,0.0000,2.5568,0.2890,0.2328,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-0.5000,6.6000,1.9000,0.0000,0.0000,-0.6813,-0.9412,-0.0306,1.7637,-1.1356,0.7065,-1.1886,0.0473,-0.4297,0.0107,-0.3988,-1.5678,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-07-20 00:00:00,-0.7973,-0.7968,-0.7987,-0.7949,-0.1784,-1.2801,6.0000,29,7.0000,3.0000,2014.0000,-0.3852,-0.1610,-0.8001,-0.7993,0.5742,-0.1242,-0.1301,0.0000,0.0000,-1.0364,1.0532,-0.6408,0.2895,-0.8114,-0.7823,0.0000,0.0000,-0.2302,-0.3016,-0.1893,-0.2437,-0.8048,-0.7912,-0.8053,-0.7971,0.0000,0.0000,-0.7853,-0.8119,0.0000,0.0000,0.0000,0.0000,0.0000,-0.6801,620.6920,602.9980,-0.2102,-0.3254,0.0009,3.2445,0.7089,0.0088,-4.1868,0.2415,-5.1691,-21.8841,0.0000,1.0000,629.9426,625.1000,0.0000,0.0000,0.0000,3.1100,0.0000,0.0000,-0.7792,-0.7743,3.0000,0.0232,7.6616,22.1225,22.0392,0.0000,0.0000,-0.7804,0.0000,0.0833,-0.3778,0.0905,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-6.0000,0.9000,1.0000,0.0000,0.0000,-0.6813,-0.9412,-0.030

In [265]:
import pandas as pd
import statsmodels.api as sm

def stepwise_selection(X, y,
                       initial_list=[],
                       threshold_in=0.05,
                       threshold_out=0.05,
                       verbose=True):
    """
    Perform stepwise selection of predictors.

    Parameters:
    X (DataFrame): Predictor variables.
    y (Series): Target variable.
    initial_list (list): Initial list of predictors.
    threshold_in (float): p-value threshold for adding variables.
    threshold_out (float): p-value threshold for removing variables.
    verbose (bool): If True, prints details of the process.

    Returns:
    list: The final list of selected predictors.
    """
    included = list(initial_list)
    while True:
        changed = False

        # Forward Selection
        excluded = list(set(X.columns) - set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(X[included + [new_column]])).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed = True
            if verbose:
                print(f"Adding {best_feature} with p-value {best_pval:.6f}")

        # Backward Elimination
        model = sm.OLS(y, sm.add_constant(X[included])).fit()
        pvalues = model.pvalues.iloc[1:]  # Exclude the intercept
        worst_pval = pvalues.max()
        if worst_pval > threshold_out:
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            changed = True
            if verbose:
                print(f"Dropping {worst_feature} with p-value {worst_pval:.6f}")

        if not changed:
            break

    return included

In [266]:
# Define the target variable
target = 'YeoJohnson_14D_Pct_Change'

# Define predictors
X = scaled_btc_data.drop(columns=[target, 'YeoJohnson_14D_Pct_Change', 'YeoJohnson_30D_Pct_Change', "YeoJohnson_7D_Pct_Change", "2D_Pct_Change", "YeoJohnson_2D_Pct_Change", 'Price', '7D_Positive', '7D_Pct_Change',  '2D_Positive', 'YeoJohnson_Change', 'Change', 'YeoJohnson_Price'], axis=1)
y = scaled_btc_data[target]


In [267]:
X = X.astype(float)

In [ ]:
# Perform stepwise selection
selected_predictors = stepwise_selection(X, y, threshold_in=0.05, threshold_out=0.05, verbose=True)

# Display the selected predictors
print("Final selected predictors:")
print(selected_predictors)

Adding 30D_Pct_Change with p-value 0.000000
Adding 14D_Pct_Change with p-value 0.000000
Dropping 30D_Pct_Change with p-value 0.190768
Adding Price_vs_MA200 with p-value 0.000000
Adding ATR_Ratio with p-value 0.000000
Adding ADX_Strength with p-value 0.000000
Adding Gold_Price with p-value 0.000000
Adding SPY_Price with p-value 0.000003
Adding MACD_cum_buy with p-value 0.000003
Adding MACD_Signal with p-value 0.000005
Adding Price_below_BB_lower with p-value 0.000032
Adding OBV with p-value 0.000043
Adding ATR with p-value 0.000058
Adding SPY_Volume with p-value 0.000051
Adding ADX_Trend_Duration with p-value 0.000719
Adding Consecutive_RSI_12_Overbought with p-value 0.000100
Adding ADX with p-value 0.001359
Dropping ADX_Strength with p-value 0.878650
Adding Year with p-value 0.002578
Adding RSI_Divergence with p-value 0.004210


In [163]:
# Select the final predictors from X
final_X = X[selected_predictors]

# Fit your model (example: linear regression)
import statsmodels.api as sm
model = sm.OLS(y, sm.add_constant(final_X)).fit()

# Summary of the final model
print(model.summary())

                               OLS Regression Results                               
Dep. Variable:     YeoJohnson_2D_Pct_Change   R-squared:                       0.014
Model:                                  OLS   Adj. R-squared:                  0.013
Method:                       Least Squares   F-statistic:                     9.320
Date:                      Sat, 26 Apr 2025   Prob (F-statistic):           3.59e-10
Time:                              11:10:18   Log-Likelihood:                -4323.9
No. Observations:                      3853   AIC:                             8662.
Df Residuals:                          3846   BIC:                             8706.
Df Model:                                 6                                         
Covariance Type:                  nonrobust                                         
                                    coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------

## Splitting the data into training, testing and validation datasets

In [ ]:
# Splitting data (70% train, 15% validation, 15% test)
train_size = int(len(final_df) * 0.7)
val_size = int(len(final_df) * 0.15)

# Time-based splits
X_train = X.iloc[:train_size]
y_train = y.iloc[:train_size]

X_val = X.iloc[train_size : train_size + val_size]
y_val = y.iloc[train_size : train_size + val_size]

X_test = X.iloc[train_size + val_size :]
y_test = y.iloc[train_size + val_size :]

NameError: name 'final_df' is not defined

## Defining functions for model performance evaluation

In [ ]:
def adj_r2_score(predictors, targets, predictions):
    r2 = r2_score(targets, predictions)
    n = predictors.shape[0]
    k = predictors.shape[1]
    return 1 - ((1 - r2) * (n - 1) / (n - k - 1))


# function to compute MAPE
def mape_score(targets, predictions):
    return np.mean(np.abs(targets - predictions) / targets) * 100


# function to compute different metrics to check performance of a regression model
def model_performance_regression(model, predictors, target):
    """
    Function to compute different metrics to check regression model performance

    model: regressor
    predictors: independent variables
    target: dependent variable
    """

    # predicting using the independent variables
    pred = model.predict(predictors)

    r2 = r2_score(target, pred)  # to compute R-squared
    adjr2 = adj_r2_score(predictors, target, pred)  # to compute adjusted R-squared
    rmse = np.sqrt(mean_squared_error(target, pred))  # to compute RMSE
    mae = mean_absolute_error(target, pred)  # to compute MAE
    mape = mape_score(target, pred)  # to compute MAPE

    # creating a dataframe of metrics
    df_perf = pd.DataFrame(
        {
            "RMSE": rmse,
            "MAE": mae,
            "R-squared": r2,
            "Adj. R-squared": adjr2,
            "MAPE": mape,
        },
        index=[0],
    )

    return df_perf

# RMSE
def rmse(predictions, targets):
    return np.sqrt(((targets - predictions) ** 2).mean())


# MAPE
def mape(predictions, targets):
    return np.mean(np.abs((targets - predictions)) / targets) * 100


# MAE
def mae(predictions, targets):
    return np.mean(np.abs((targets - predictions)))

# To see the feature importance of variables in the final model
def feature_importances(model, feature_names, n=10):
    if isinstance(model,LinearRegression):
        importances = model.coef_
    else:
        importances = model.feature_importances_
    zipped = sorted(zip(feature_names, importances), key=lambda x: -x[1])
    for i, f in enumerate(zipped[:n]):
        print("%d: Feature: %s, %.3f" % (i+1, f[0], f[1]))


# Model Performance on test and train data
def model_pref(olsmodel, x_train, x_test, y_train,y_test):

    # Insample Prediction
    y_pred_train = olsmodel.predict(x_train)
    y_observed_train = y_train

    # Prediction on test data
    y_pred_test = olsmodel.predict(x_test)
    y_observed_test = y_test

    print(
        pd.DataFrame(
            {
                "Data": ["Train", "Test"],
                "RMSE": [
                    rmse(y_pred_train, y_observed_train),
                    rmse(y_pred_test, y_observed_test),
                ],
                "MAE": [
                    mae(y_pred_train, y_observed_train),
                    mae(y_pred_test, y_observed_test),
                ],
                "MAPE": [
                    mape(y_pred_train, y_observed_train),
                    mape(y_pred_test, y_observed_test),
                ],
            }
        )
    )

##Model 1: Linear Regression

In [ ]:
predictors = valid_names

# Create the formula for the regression model
formula = f"{target} ~ {' + '.join(predictors)}"

# Fit the OLS regression model
lm_model = smf.ols(formula=formula, data= scaled_btc_data_with_interactions).fit()

# Print the summary of the model
print(lm_model.summary())

                              OLS Regression Results                             
Dep. Variable:     YeoJohnson_Pct_Change   R-squared:                       0.287
Model:                               OLS   Adj. R-squared:                  0.276
Method:                    Least Squares   F-statistic:                     25.53
Date:                   Wed, 23 Apr 2025   Prob (F-statistic):          3.43e-243
Time:                           18:09:28   Log-Likelihood:                -5094.9
No. Observations:                   4062   AIC:                         1.032e+04
Df Residuals:                       3998   BIC:                         1.072e+04
Df Model:                             63                                         
Covariance Type:               nonrobust                                         
                                             coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------

### Checking Model Assumptions

Model assumption 1: Mean of residuals should be 0

Model assumption 2: No Heteroscedasticity

Model assumption 3: Linearity

Model assumption 4: Normality of the residuals

###Checking for Autocorrelation

###Checking for Multicollinearity

PCA

Lasso/ Ridge?

##Model 2: ANN

In [ ]:
import numpy as np
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error

def ann_lin(x_train, y_train, x_test, y_test,
            size, max_iter=100, trace=False, n_try=5):
    best_train_mse = np.inf
    best_test_mse  = np.inf
    best_train_model = None
    best_test_model  = None

    for i in range(n_try):
        # create a regressor with one hidden layer of 'size' units
        model = MLPRegressor(hidden_layer_sizes=(size,),
                             activation='relu',
                             solver='adam',
                             max_iter=max_iter,
                             random_state=i)

        model.fit(x_train, y_train)

        # compute train MSE
        y_train_pred = model.predict(x_train)
        train_mse = mean_squared_error(y_train, y_train_pred)

        # compute test MSE
        y_test_pred = model.predict(x_test)
        test_mse = mean_squared_error(y_test, y_test_pred)

        if trace:
            print(f"Run {i+1}/{n_try} → train MSE: {train_mse:.4f}, test MSE: {test_mse:.4f}")

        # update best-by-train
        if train_mse < best_train_mse:
            best_train_mse = train_mse
            best_train_model = model

        # update best-by-test
        if test_mse < best_test_mse:
            best_test_mse = test_mse
            best_test_model = model

    return {
        'best_train_mse': best_train_mse,
        'best_test_mse':  best_test_mse,
        'best_train_model': best_train_model,
        'best_test_model':  best_test_model
    }


##Model 3: Random Forest

Decision Tree (5104)

In [ ]:
from sklearn.tree import DecisionTreeRegressor, plot_tree

# Build decision tree
dt_model = DecisionTreeRegressor(max_depth=3, random_state=1)
dt_model.fit(X_train, y_train)

# Visualize decision tree
plt.figure(figsize=(18, 10))
plot_tree(dt_model, feature_names=X_train.columns, filled=True, rounded=True, fontsize=10)
plt.title('Basic Decision Tree for Bitcoin Price', fontsize=15)
plt.tight_layout()
plt.show()

dt_val_performance = model_performance_regression(dt_model, X_val, y_val)
print("Basic Decision Tree performance:", dt_val_performance)

Random Forest

In [ ]:
# Build random forest model
regressor = RandomForestRegressor(n_estimators=100, random_state=1)
regressor.fit(X_train, y_train)


In [ ]:
# Evaluate on validation set
val_performance = model_performance_regression(regressor, X_val, y_val)
print("Validation performance:", val_performance)

###Feature Selection

In [ ]:
from sklearn.inspection import permutation_importance
# Calculate permutation importance on validation set
perm_result = permutation_importance(
    regressor, X_val, y_val,
    n_repeats=10,
    random_state=42,
    n_jobs=-1
)

# Get importance scores and sort them
importances = perm_result.importances_mean
indices = np.argsort(importances)[::-1]  # Sort in descending order

In [ ]:
# Plot feature importances
plt.figure(figsize=(12, 8))
plt.title('Feature Importance (Permutation Method)', fontsize=15)
plt.bar(range(len(indices)), importances[indices], color='darkgreen')
plt.xticks(range(len(indices)), [X_train.columns[i] for i in indices], rotation=90)
plt.tight_layout()
plt.show()

In [ ]:
# Select features with positive importance (threshold can be adjusted)
threshold = 0  # Only include features that improve predictions
selected_features = [X_train.columns[i] for i in indices if importances[i] > threshold]
print(f"Selected {len(selected_features)} out of {X_train.shape[1]} features")
print("Selected features:", selected_features)

In [ ]:
# Create datasets with only selected features
X_train_selected = X_train[selected_features]
X_val_selected = X_val[selected_features]
X_test_selected = X_test[selected_features]

###Model Validation and Hyperparameter Tuning

In [ ]:
# Hyperparameter tuning
rf_tuned = RandomForestRegressor(random_state=1)

# Grid of parameters to choose from
parameters = {
    "n_estimators": [100, 200, 300],
    "max_depth": [None, 5, 10, 15],
    "max_features": ["sqrt", "log2", 0.8, 1],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4]
}

'''parameters = {
    "n_estimators": [110, 120],
    "max_depth": [5, 7],
    "max_features": [0.8, 1]
}'''

# Use time series cross-validation
tscv = TimeSeriesSplit(n_splits=5)

# Run the grid search on selected features
grid_obj = GridSearchCV(rf_tuned, parameters, scoring='r2', cv=tscv)
grid_obj.fit(X_train_selected, y_train)  # Use selected features

# Get the best model
rf_tuned_regressor = grid_obj.best_estimator_
print("Best parameters:", grid_obj.best_params_)
tuned_val_performance = model_performance_regression(rf_tuned_regressor, X_val_selected, y_val)
print("Tuned model validation performance:", tuned_val_performance)

In [ ]:
# Compare base model vs tuned model on validation
print("\nModel comparison on validation set:")
print(f"Base model R²: {val_performance['R²']:.4f}")
print(f"Tuned model R²: {tuned_val_performance['R²']:.4f}")

In [ ]:
# Train final model on combined training and validation data
X_train_val = pd.concat([X_train_selected, X_val_selected])
y_train_val = pd.concat([y_train, y_val])

final_rfmodel = RandomForestRegressor(**grid_obj.best_params_, random_state=1)
final_rfmodel.fit(X_train_val, y_train_val)

In [ ]:
# Final evaluation on test set only once
final_test_performance = model_performance_regression(final_rfmodel, X_test_selected, y_test)
print("\nFinal test performance:", final_test_performance)

In [ ]:
# Plot actual vs predicted values on test set
y_pred = final_rfmodel.predict(X_test_selected)
plt.figure(figsize=(14, 6))
plt.title('Bitcoin Price: Actual vs Predicted (Selected Features)', fontsize=15)
plt.plot(y_test.index, y_test, label='Actual', color='blue')
plt.plot(y_test.index, y_pred, label='Predicted', color='red', linestyle='--')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Bitcoin Price % Change', fontsize=12)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Example prediction with the model
prediction = final_rfmodel.predict(X_test_selected)[0]  #the first prediction in test set

mean_preds, lower_bounds, upper_bounds = predict_with_confidence(final_rfmodel, X_test_selected)
margin = (upper_bounds[0] - lower_bounds[0]) / 2

print(f"Predicted Bitcoin % change: {mean_preds[0]:.2f}% ± {margin:.2f}% with 95% confidence")

#Model 4: Logistic Regression for 5104

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, accuracy_score

# Create binary target
y_binary = (y > 0).astype(int)

X_train_val, X_test = train_test_split(X, test_size=0.2, shuffle=False)
y_train_val_binary, y_test_binary = train_test_split(y_binary, test_size=0.2, shuffle=False)
X_train, X_val = train_test_split(X_train_val, test_size=0.25, shuffle=False)
y_train_binary, y_val_binary = train_test_split(y_train_val_binary, test_size=0.25, shuffle=False)

#Model Performance Comparison

#Final Model Evaluation and Backtesting

Walk-Forward Validation

## Back Transformation After prediction (for interpretation)

In [ ]:
# Recreate PowerTransformers with stored lambdas for back-transformation
power_transformer_price.lambdas_ = [lambdas['Price_lambda']]
power_transformer_change.lambdas_ = [lambdas['Change_lambda']]
power_transformer_7d_pct_change.lambdas_ = [lambdas['7D_Pct_Change_lambda']]
power_transformer_2d_pct_change.lambdas_ = [lambdas['2D_Pct_Change_lambda']]

# Step 1: De-standardize the standardized values to get Yeo-Johnson transformed values
scaled_btc_data['YeoJohnson_Price_Back'] = (
    scaled_btc_data['YeoJohnson_Price'] * transformed_stds['Price_std']
) + transformed_means['Price_mean']

scaled_btc_data['YeoJohnson_Change_Back'] = (
    scaled_btc_data['YeoJohnson_Change'] * transformed_stds['Change_std']
) + transformed_means['Change_mean']

scaled_btc_data['YeoJohnson_7D_Pct_Change_Back'] = (
    scaled_btc_data['YeoJohnson_7D_Pct_Change'] * transformed_stds['7D_Pct_Change_std']
) + transformed_means['7D_Pct_Change_mean']

scaled_btc_data['YeoJohnson_2D_Pct_Change_Back'] = (
    scaled_btc_data['YeoJohnson_2D_Pct_Change'] * transformed_stds['2D_Pct_Change_std']
) + transformed_means['2D_Pct_Change_mean']

# Step 2: Create temporary DataFrames with the exact column names used during fit
price_back_df = scaled_btc_data[['YeoJohnson_Price_Back']].rename(columns={'YeoJohnson_Price_Back': 'Price'})
change_back_df = scaled_btc_data[['YeoJohnson_Change_Back']].rename(columns={'YeoJohnson_Change_Back': 'Change'})
pct_change_7d_back_df = scaled_btc_data[['YeoJohnson_7D_Pct_Change_Back']].rename(columns={'YeoJohnson_7D_Pct_Change_Back': '7D_Pct_Change'})
pct_change_2d_back_df = scaled_btc_data[['YeoJohnson_2D_Pct_Change_Back']].rename(columns={'YeoJohnson_2D_Pct_Change_Back': '2D_Pct_Change'})

# Step 3: Apply the inverse transformation to recover the original values
scaled_btc_data['Original_Price'] = power_transformer_price.inverse_transform(price_back_df)
scaled_btc_data['Original_Change'] = power_transformer_change.inverse_transform(change_back_df)
scaled_btc_data['Original_7D_Pct_Change'] = power_transformer_7d_pct_change.inverse_transform(pct_change_7d_back_df)
scaled_btc_data['Original_2D_Pct_Change'] = power_transformer_2d_pct_change.inverse_transform(pct_change_2d_back_df)

# Step 4: Display the back-transformed data
print("\nBack-Transformed Data:")
print(scaled_btc_data[['Original_Price', 'Original_Change', 'Original_7D_Pct_Change', 'Original_2D_Pct_Change']].head())

NameError: name 'scaled_btc_data' is not defined